In [ ]:
import helper as hlp

import numpy as np
from numpy.lib.recfunctions import drop_fields

import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
import matplotlib.animation as animation
%matplotlib inline

import scipy.interpolate as sci
import scipy.optimize as sco
import scipy.stats as scs
import scipy.integrate as scint

import json
import time
import datetime
import pickle
from astropy.time import Time as astrotime
from tqdm import tqdm_notebook as tqdm

import sklearn.neighbors as skn
import sklearn.model_selection as skms  # Newer version of grid_search
from sklearn.utils import check_random_state

from anapymods3.healpy import wrap_theta_phi_range
from anapymods3.plots.astro import skymap, EquCoordsToMapCoords
from anapymods3.plots import split_axis, get_binmids, hist_marginalize, dg
from anapymods3.stats.sampling import rejection_sampling
from anapymods3.stats import KDE, json2kde

# Some globals
hoursindays = 24.
secinday = hoursindays * 60. * 60. 

# Load data

Load IC86 data from epinat, which should be the usual IC86-I (2011) PS sample, but pull corrected and OneWeights corrected by number of events generated.

In [ ]:
exp = np.load("data/IC86_I_data.npy")
mc = np.load("data/IC86_I_mc.npy")
# Use the officially stated livetime, not the ones from below
livetime = 332.61

# Get data livetime

Generate from good run list as stated here:
- http://icecube.wisc.edu/~coenders/html/build/html/ic86-bdt/muonL3.html
- https://wiki.icecube.wisc.edu/index.php/IC86_I_Point_Source_Analysis/Data_and_Simulation

It should be 332.61 days as stated by jefeintzeig and scoenders.
We create one bin per included run, with exactly that width.
Excluded runs are those with too high/low rate and without everything marked "good".

Livetime ist a bit higher, because we used a newer runlist from iclive instead of the old non-json v1.4.
See side test for that comparison.

Problem is also, that this runlist includes runs with zero events, so they are probably cut out due to the old runlist in the original selection.

In [ ]:
def filter_runs(run):
    """
    Filter runs as stated in jfeintzig's doc.
    """
    exclude_runs = [120028, 120029, 120030, 120087, 120156, 120157]
    if ((run["good_i3"] == True) & (run["good_it"] == True) &
        (run["run"] not in exclude_runs)):
        return True
    else:
        return False

In [ ]:
goodrun_dict, _livetime = hlp.create_goodrun_dict(
    runlist="data/runlists/ic86-i-goodrunlist.json", filter_runs=filter_runs)

# We don't use this livetime, but the "official" one from jfeintzeig's page
print("IC86-I livetime from iclive: ", _livetime)

# Bin BG according to runlist

Each run is one bin in the bg rate vs time plot.
The rate is normed to Hertz by dividing through the bin sizes in seconds.

In [ ]:
h = hlp._create_runtime_bins(exp["timeMJD"], goodrun_dict=goodrun_dict,
                             remove_zero_runs=True)

In [ ]:
# Plot runs
fig, ax = plt.subplots(1, 1)

start, stop = h["start_mjd"], h["stop_mjd"]
rate = h["rate"]
xerr = 0.5 * (stop - start)
yerr = h["rate_std"]
binmids = 0.5 * (stop + start)
ax.errorbar(binmids, rate, xerr=xerr, yerr=yerr, fmt=",")

# Setup main axis
ax.set_xlim(start[0], stop[-1])
ax.set_ylim(0, None)
ax.set_xlabel("MJD")
ax.set_ylabel("Rate in Hz")
# Rotate bottom labels if needed
# def xlabels(x):
#     return ["{:5d}".format(int(xi)) for xi in x]
# ax.set_xticklabels(xlabels(ax.get_xticks()), rotation=60,
#                    horizontalalignment="right")

# Second xaxis on top with month and year.
# Convert MJD to datetimes, make dates for every month and convert to mjd
# http://stackoverflow.com/questions/22696662/ \
#   python-list-of-first-day-of-month-for-given-period
datetimes = astrotime(binmids, format="mjd").to_datetime()
dt, end = datetimes[0], datetimes[-1]
datetimes_ticks = []
while dt < end:
    if not dt.month % 12:
        dt = datetime.datetime(dt.year + 1, 1, 1)
    else:
        dt = datetime.datetime(dt.year, dt.month + 1, 1)
    datetimes_ticks.append(dt)
mjd_ticks = astrotime(datetimes_ticks, format="datetime").mjd

# New axis on top, make sure, we use the same range
ax2 = ax.twiny()
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(mjd_ticks)
ax2.set_xticklabels([dtt.strftime("%b '%y") for dtt in datetimes_ticks],
                    rotation=60, horizontalalignment="left")

fig.tight_layout()
plt.show()

# Time dependent rate function

Rate ist time dependent because of seasonal variation.
We take this varariation into account by fitting a priodic function to the time resolved rate.

The data is built by calculating the rate in each run as seen before.
This rate is correctly normalized and smoothes local fluctuations.

### Peridoc function with a weighted least squares fit

See side_test for comparison to spline fits.
The function is a simple sinus scalable by 4 parameters to fit the shape of the rates:

$$
    f(x) = a\cdot \sin(b\cdot(x - c)) + d
$$

The least squares loss function is

$$
    R = \sum_i (w_i(y_i - f(x_i)))^2
$$

Weights usually are standard deviations from each data point.
But here we have a counting experiment for each run and we want to give higher statistics runs a higher weight so we shouldn't use Poisson errors directly as they scale with the number of counts.
The relative error instead drops with higher count rate so we use the inverse of the relative error to weight each point.

$$
    w_i = \frac{N}{\sqrt{N}} = \sqrt{N}
$$

Also this has the nice property of excluding zero entry bins.

Seed values are estimated from plot rate vs time.

- Period should be 365 days (MJD) because we have one year of data so we choose $b0 = 2\pi/365$.
- Amplitude is about $a_0=-0.0005$, because sinus seems to start with negative values.
- The x-offset is choose as the first start date, to get the right order of magnitude.
- The y-axis intersection $d$ schould be close to the weighted average, so we take this as a seed.

The bounds are motivated as follows (and if we don't hit them, it's OK to use them).

- Amplitude $a$ should be positive, this also resolves a degenracy between a-axis offset.
- The period $b$ should scatter around one year, a period larger than +-1 half a year is unphysical.
- The x-offset $c$ cannot be greater than the initial +- the period because we have a periodic function.
- The y-axis offset $d$ is arbitrarily constrained, but as seen from the plot it should not exceed 0.1. 

In [ ]:
def f(x, pars):
    """
    Returns the rate at a given time in MJD.
    """
    a, b, c, d = pars
    return a * np.sin(b * (x - c)) + d

def lstsq(pars, *args):
    """
    Weighted leastsquares min sum((wi * (yi - fi))**2)
    """
    # data x,y-values and weights are fixed
    x, y, w = args
    _f = f(x, pars)
    return np.sum((w * (y - _f))**2)

In [ ]:
# Seed values from consideration above.
# a0 = -0.0005
# b0 = 2. * np.pi / 365.  # We could restrict the period to one yr exact.
# c0 = np.amin(start_mjd)
# d0 = np.average(h, weights=yerr**2)

rate = h["rate"]
rate_std = h["rate_std"]
X = exp["timeMJD"]
binmids = 0.5 * (h["start_mjd"] + h["stop_mjd"])

a0 = 0.5 * (np.amax(rate) - np.amin(rate))
b0 = 2. * np.pi / 365.
c0 = np.amin(X)
d0 = np.average(rate, weights=rate_std**2)

x0 = [a0, b0, c0, d0]
# Bounds as explained above
bounds = [[None, None], [0.5 * b0, 1.5 * b0], [c0 - b0, c0 + b0, ], [0, 0.01]]
# x, y values, weights
args = (binmids, rate, np.sqrt(rate_std))

res = sco.minimize(fun=lstsq, x0=x0, args=args, bounds=bounds)
bf_pars = res.x

print("Amplitude   : {: 13.5f} in Hz".format(res.x[0]))
print("Period (d)  : {: 13.5f} in days".format(2 * np.pi / res.x[1]))
print("Offset (MJD): {: 13.5f} in MJD".format(res.x[2]))
print("Avg. rate   : {: 13.5f} in Hz".format(res.x[3]))

In [ ]:
# Define the rate function:
def rate_fun(t):
    """
    Returns the rate at a given time in MJD.
    
    Parameters
    ----------
    t : array-like
        Time in MJD.
        
    Returns
    -------
    rate : array-like
        The rate of background events in Hz.
    """
    return f(t, res.x)

In [ ]:
# Plot runs
start, stop = h["start_mjd"], h["stop_mjd"]
rate = h["rate"]
xerr = 0.5 * (stop - start)
yerr = h["rate_std"]
binmids = 0.5 * (stop + start)

plt.errorbar(binmids, rate, xerr=xerr, yerr=yerr, fmt=",")
plt.ylim(0, None);

# Plot fit
t = np.linspace(start[0], stop[-1], 1000)
y = rate_fun(t)
plt.plot(t, y, zorder=5)

# Plot y shift dashed to see baseline or years average
plt.axhline(bf_pars[3], 0, 1, color="C1", ls="--", label="")

plt.xlim(start[0], stop[-1])
plt.xlabel("MJD")
plt.ylabel("Rate in Hz")

# plt.savefig("./data/figs/time_rate_sinus.png", dpi=200)
plt.ylim(0, 0.009)
plt.tight_layout()
plt.show()

# Draw Number of Background Events 

The fitted spline models the expected background at a given time.
Draw the actual number of events to inject per BG trial within a given time window using a poisson distribution with the mean from the spline.

Classically the events drawn are then assigned a random time within the time window.
But as we have the rate function, we can sample times from that function using a rejection sampling.
This will only affect larger intervals, where the curvature can be seen and should only be used in these cases, as it is more costly to sample.

## Sample number of events in frame

Sample for single source but many trials at once.

In [ ]:
# Execute "Time dependent rate function" cells for best fit result
best_pars = res.x

def _transform_trange_mjd(t, trange):
    """
    Returns
    -------
    trange : array-like, shape(2, len(t))
        Reshaped time window `[[t0], [t1]]` in MJD for each time t.
    """
    t = np.atleast_1d(t)
    return t + np.array(trange).reshape(2, 1) / secinday

def rate_fun_integral(t, trange):
    """
    Analytic integral of the sinusodial rate function in interval `trange`.

    t, trange: float, [float, float]
        Time in MJD and tine frame around t in seconds.
    """
    a, b, c, d = res.x

    # Transform time window to MJD
    t0, t1 = _transform_trange_mjd(t, trange)
    # Split analytic expression for readability only
    per = a / b * (np.cos(b * (t0 - c)) - np.cos(b * (t1 - c)))
    lin = d * (t1 - t0)
    # Match units with secinday = 24 * 60 * 60 s/MJD = 86400 / (Hz*MJD)
    #     [a], [d] = Hz, [b], [c], [ti] = MJD
    #     [a / b] = Hz * MJD, [d * (t1 - t0)] = HZ * MJD
    return (per + lin) * secinday
    
# Just choose a single time and a large time frame to see the sinus shape
t = h["start_mjd"][100]
trange = [0, 365 * secinday]  # 1 year window -> 1 period

# Expectation is the integrals over the time frame
expect = rate_fun_integral(t, trange)

# Compare expectation with rectangular integration rule dt * avg
print("Expectation : ", expect)
print("Estimated   : ", np.diff(trange) * res.x[3])

# Sample single number of events from poisson distribution
ntrials = 100
nevents = np.random.poisson(lam=expect, size=ntrials)
print("Sampled events        : ", sum(nevents))
print("Sample mean rate in Hz: ", sum(nevents) / ntrials / np.diff(trange)[0])
print("Fitted mean rate in Hz: ", bf_pars[3])

## Now the sampling of random times in the time frame

From the number of events, make injected times sampled form the rate function per trial.
Sample all events at once and split up in trials later.
This is way faster than looping over every trial.

In [ ]:
def sample_times(t, trange, n_samples=1):
    """
    %(RateFunction.sample.summary)s

    Parameters
    ----------
    %(RateFunction.sample.parameters)s

    Returns
    -------
    %(RateFunction.sample.returns)s
    """
    # rejection_sampling needs bounds in shape (1, 2)
    trange = _transform_trange_mjd(t, trange).T
    times = rejection_sampling(rate_fun, bounds=trange, n=n_samples)[0]
    return times

In [ ]:
# Sample all events for a time frame trange at once and split later
tot_nevts = np.sum(nevents)
print("Sampling total events : ", tot_nevts)

times = sample_times(t, trange, tot_nevts)
print(times)

# Build index slices
start = np.cumsum(np.append(0, nevents[:-1]))
end = np.cumsum(nevents)
# Split them up in list of trials, so each trial has the correct num of times
trials = [times[i:k] for i, k in zip(start, end)]

# See, if each trial has the corret amount of events sampled
print("\nTrials have correct length: ", np.all(np.equal(np.array(
                list(map(len, trials))), nevents)))

print("nevents per trial:\n", nevents[:10].reshape(10, 1))
print("Examples of sampled times per trial:\n", trials[:5])

In [ ]:
# Plot samples trials and see that it follows a sinus function
h_, b_ = np.histogram(times, bins=50, density=False)
err = np.sqrt(h_)

# Norm to rate
norm = np.diff(b_) * secinday * ntrials
h_ = h_ / norm
err = err / norm

plt.errorbar(get_binmids(b_), h_, xerr=0.5 * np.diff(b_), yerr=err,
             fmt="none", markersize=3)

x_ = t + np.linspace(trange[0] / secinday, trange[1] / secinday, 500)
plt.plot(x_, rate_fun(t=x_))

plt.xlabel("MJD")
plt.ylabel("Rate in Hz")

plt.tight_layout()
plt.show()

# Create the BG PDF from data

Proceeding to section 6.3.1 Randomized BG Injection, p. 113.
Mrichmann draws events by:

1. Get number of bg events to be injected from a poisson distribution with expectation values drawn from the previously build bg temporal distribution.

   $$
   P_{\langle n_B\rangle}(N_m) = \frac{\langle n_B\rangle^{N_m}}{N_m\!}\cdot \exp(\langle n_B\rangle)
   $$
   
2. These events are then drawn from a 3D pdf in energy proxy, zenith proxy and sigma proxy.
   He does it by dividing 10x10x10 bins, first selecting energy, then zenith in that energy bin, then sigma in that zenith bin.
   
Here we create a smooth PDF using a kernel density estimator and obtain a sample by running a MCMC chain to create a sample a priori.
The bandwidth is set globally and cross validated to be robust.

**Some note on `numpy.histogramdd`:**

The input must be an array with shape (nDim, len(data)).

Shape of h is the same as the number of bins in each dim: (50, 40, 10)
So the first dimension picks a single logE slice -> h[i].shape = (40, 10)
Second dim picks a dec slice -> h[:, i].shape = (50, 10)
3rd picks a sigma slice -> h[:, :, i].shape = (50, 40)

This is important: meshgrid repeats in second axis on first array xx.
For the second array, the first axis is repeated.
But h iterates over energy in 1st axis. So if we don't transpose, we have the whole histogram flipped! Compare to plot in mrcihmanns thesis (cos(zen))

**Some notes on KDE:**

Sebastian has already made a tool for adaptive and asymmetric KDE.
1. The Kernel is the covariance matrix of the whole data set to regard different scales
    + Note: This may only be a problem, if one dim is spread with peaks, while the other is wide spread only. Then we cannot scale the Kernel to small to fit the peaks because the smooth dimension is preventing that.
2. Use Silvermans or Scotts rule as a first guess.
3. Run a second pass and vary the local bandwidth according to the first guess local density.

We could replace 1 and 2 by scaling the data with the inverse covariance and then using a cross validation to find the first guess bandwidth.
Then using a second pass to vary locally.

## 3D histogram of BG data
First we make a 3D histogram to better compare to mrichmann and to get an overview over the distribution.

In [ ]:
# Cut sigmas in the sample to obtain smooth tail from KDE and remove outliers
m = exp["sigma"] < np.deg2rad(10)
_logE = exp["logE"][m]
_dec = exp["dec"][m]
_sigma = exp["sigma"][m]
# Sample must match with the one used in training here
sample = np.vstack((_logE, _dec, _sigma)).T

# Binning is rather arbitrary because we don't calc stuff with the hist
bins = [50, 50, 50]

# Plot in degrees and in sinDec
_sam = np.vstack((_logE, np.sin(_dec), np.rad2deg(_sigma))).T

h, bins = np.histogramdd(sample=_sam, bins=bins, normed=False)

# Make a nice corner plot
label = ["logE", "sinDdec", "sigma deg"]
fig, ax = hlp.corner_hist(h, bins=bins,
                          label=label,
                          hist2D_args={"cmap": "inferno", "norm": LogNorm()},
                          hist_args={"color":"C1", "alpha": 0.5, "log": True})

# plt.savefig("./data/figs/bg_corner_scaled.png", dpi=200)

## Kernel Density Estimation

Use adaptive width KDE to describe BG data and be able to smoothly draw new events from it.
We fitted one set of params to the full data and stored it to avoid lengthy (~60 mins.) refitting when testing.
A optimal set of parameters gets determined in a cross validation.

In [ ]:
# Assign model from CV, which has already evaluated adaptive kernels
kde_inj = json2kde("data/awKDE_CV/CV10_glob_bw_alpha_EXP_IC86I_" +
                   "CUT_sig.ll.20_PARS_diag_True_pass2.json")

# Sample with bounds, because gaussian KDEs have no border by default
bounds = np.array([[None, None], [-np.pi / 2. , np.pi / 2.], [0, None]])
n_samples = int(1e6)
kde_sam = kde_inj.sample(n_samples)

# Plot in degrees and in sinDec
_sam_kde = np.vstack((kde_sam[:, 0],
                      np.sin(kde_sam[:, 1]),
                      np.rad2deg(kde_sam[:, 2]))).T

bins = [np.linspace(1, 7, 50), np.linspace(-1, 1,50), np.linspace(0, 10, 50)]
h, _ = np.histogramdd(sample=_sam_kde, bins=bins, normed=False)
fig, ax = hlp.corner_hist(h, bins=bins,
                         label=label,
                         hist2D_args={"cmap": "inferno", "norm": LogNorm()},
                         hist_args={"color":"C1", "alpha": 0.5, "log": True})

# plt.savefig("./data/figs/bg_corner_scaled.png", dpi=200)
plt.show()

## Compare KDE to original data

Make a ratio histogram of the KDE sample and the original data sample.

### 2D marginalization

In [ ]:
# Create 2D hists, by leaving out one parameter
xlabel = [label[0], label[0], label[1]]
ylabel = [label[1], label[2], label[2]]

for i, axes in enumerate([[0, 1], [0, 2], [1, 2]]):
    _b = np.array(bins)
    h_exp, b_exp = np.histogramdd(_sam[:, axes],
                                  bins=_b[axes], normed=True)
    h_kde, b_kde = np.histogramdd(_sam_kde[:, axes],
                                  bins=_b[axes], normed=True)
    
    # KDE is expectation, but sampled with much more events.
    # Weights would simply scale the total number of KDE events to match the
    # number of original events. That would be the mean for the poisson
    # distribution in each bin. So to get OK KDE expectation sqrt(n) errors
    # in each bin, we divide not by the number of drawn KDE but by the number
    # of original events.   
    # Again shapes of meshgrid and hist are transposed
    diffXX, _ = np.meshgrid(np.diff(_b[0]), np.diff(_b[1]))
    norm_kde = len(exp) * diffXX.T
    sigma_kde = np.sqrt(h_kde / norm_kde)

    # Make 3 different diff/ratio hists to estimate KDE quality in
    # 1D marginalization.
    m = (h_exp > 0.)
    ratio_h = np.zeros_like(h_exp)
    ratio_h[m] = h_kde[m] / h_exp[m]

    diff_h = h_kde - h_exp

    m = (sigma_kde > 0.)
    sigma_ratio_h = np.zeros_like(h_exp)
    sigma_ratio_h[m] = (h_exp[m] - h_kde[m]) / sigma_kde[m]

    # Bin mids and hist grid
    _b = b_exp
    m = get_binmids(_b)
    xx, yy = map(np.ravel, np.meshgrid(m[0], m[1]))
    
    
    # Big plot on the left and three right
    fig = plt.figure(figsize=(10, 6))
    gs = gridspec.GridSpec(3, 3)
    axl = fig.add_subplot(gs[:, :2])
    axrt = fig.add_subplot(gs[0, 2])
    axrc = fig.add_subplot(gs[1, 2])
    axrb = fig.add_subplot(gs[2, 2])
    
    # Steal space for colorbars
    caxl = split_axis(axl, "right")
    caxrt = split_axis(axrt, "left")
    caxrc = split_axis(axrc, "left")
    caxrb = split_axis(axrb, "left")

    # Unset top and center xticklabels as they are shared with the bottom plot
    axrt.set_xticklabels([])
    axrc.set_xticklabels([])
        
    # Left: Difference over KDE sigma
    # cbar_extr = max(np.amax(sigma_ratio_h),  # Center colormap to min/max
    #                         abs(np.amin(sigma_ratio_h)))
    _, _, _, imgl = axl.hist2d(xx, yy, bins=_b, weights=sigma_ratio_h.T.ravel(),
                               cmap="seismic", vmax=5, vmin=-5)
    cbarl = plt.colorbar(cax=caxl, mappable=imgl)
    axl.set_xlabel(xlabel[i])
    axl.set_ylabel(ylabel[i])
    axl.set_title("(exp - kde) / sigma_kde")
    
    # Right top: Ratio
    _, _, _, imgrt = axrt.hist2d(xx, yy, bins=_b, weights=ratio_h.T.ravel(),
                                 cmap="seismic", vmax=2, vmin=0);
    cbarrt = plt.colorbar(cax=caxrt, mappable=imgrt)
    axrt.set_title("kde / exp")

    # Right center: Data hist
    _, _, _, imgrc = axrc.hist2d(xx, yy, bins=_b, weights=h_exp.T.ravel(),
                                 cmap="inferno", norm=LogNorm());
    cbarrc = plt.colorbar(cax=caxrc, mappable=imgrc)
    axrc.set_title("exp logscale")

    # Right bottom: KDE hist, same colorbar scale as on data
    _, _, _, imgrb = axrb.hist2d(xx, yy, bins=_b, weights=h_kde.T.ravel(),
                                 cmap="inferno", norm=LogNorm());
    # Set with same colormap as on data
    imgrb.set_clim(cbarrc.get_clim())
    cbarrb = plt.colorbar(cax=caxrb, mappable=imgrb)
    axrb.set_title("kde logscale")
    
    # Set tick and label positions
    for ax in [caxrt, caxrc, caxrb]:
        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_left()
    
    fig.tight_layout()
#     plt.savefig("./data/figs/kde_data_2d_{}_{}.png".format(
#         xlabel[i], ylabel[i]), dpi=200)
    plt.show()

### 1D marginalization

In [ ]:
# Pseudo smooth marginalization is done by sampling many point from KDE an
# using a finely binned 1D histogram, so it looks smooth
xlabel = label

for i, axes in enumerate([0, 1, 2]):
    _b = np.array(bins)
    h_exp, b_exp = np.histogram(_sam[:, axes],
                                bins=_b[axes], normed=True)
    h_kde, b_kde = np.histogram(_sam_kde[:, axes],
                                bins=_b[axes], normed=True)
    
#     h_exp, b_exp = hist_marginalize(h, bins, axes=axes)
#     h_kde, b_kde = hist_marginalize(bg_h, bg_bins, axes=axes)
      
    # KDE errorbars as in 2D case
    norm_kde = len(exp) * np.diff(b_kde)
    sigma_kde = np.sqrt(h_kde / norm_kde)

    # Make 3 different diff/ratio hists to estimate KDE quality in
    # 1D marginalization.
    m = (h_exp > 0.)
    ratio_h = np.zeros_like(h_exp)
    ratio_h[m] = h_kde[m] / h_exp[m]

    diff_h = h_kde - h_exp

    m = (sigma_kde > 0.)
    sigma_ratio_h = np.zeros_like(h_exp)
    sigma_ratio_h[m] = (h_exp[m] - h_kde[m]) / sigma_kde[m]

    # Bin mids
    _b = b_exp
    m = get_binmids([_b])[0]
    
    # Plot both and the ration normed. Big plot on the left and three right
    fig = plt.figure(figsize=(10, 6))
    gs = gridspec.GridSpec(3, 3)
    axl = fig.add_subplot(gs[:, :2])
    axrt = fig.add_subplot(gs[0, 2])
    axrc = fig.add_subplot(gs[1, 2])
    axrb = fig.add_subplot(gs[2, 2])

    axrt.set_xticklabels([])
    axrc.set_xticklabels([])

    # Set ticks and labels right
    for ax in [axrt, axrc, axrb]:
        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_right()

    # Limits
    for ax in [axl, axrt, axrc, axrb]:
        ax.set_xlim(_b[0], _b[-1])
        
    # Main plot:
    # Plot more dense to mimic a smooth curve
    __h, __b = np.histogram(_sam_kde[:, i], bins=200,
                            range=[_b[0], _b[-1]], density=True)
    __m = get_binmids([__b])[0]
    axl.plot(__m, __h, lw=3, alpha=0.5)
    
    _ = axl.hist(m, bins=_b, weights=h_exp, label="exp", histtype="step",
                 lw=2, color="k")
    _ = axl.errorbar(m, h_kde, yerr=sigma_kde, fmt=",", color="r")
    _ = axl.hist(m, bins=_b, weights=h_kde, label="kde", histtype="step",
                 lw=2, color="r")    
    
    axl.set_xlabel(xlabel[i])
    axl.legend(loc="upper right")

    # Top right: Difference
    _ = axrt.axhline(0, 0, 1, color="k", ls="-")
    _ = axrt.hlines([-.02, -.01, .01, .02], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrt.hist(m, bins=_b, weights=diff_h, histtype="step", lw=2, color="r")
    axrt.set_ylim(-.05, +.05)
    axrt.set_ylabel("kde - exp")

    # Center right: Ratio
    _ = axrc.axhline(1, 0, 1, color="k", ls="-")
    _ = axrc.hlines([0.8, 0.9, 1.1, 1.2], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrc.hist(m, bins=_b, weights=ratio_h, histtype="step", lw=2, color="r")
    axrc.set_ylim(.5, 1.5)
    axrc.set_ylabel("kde / exp")

    # Bottom right: Ratio of diff to sigma of expectation
    _ = axrb.axhline(0, 0, 1, color="k", ls="-")
    _ = axrb.hlines([-2, -1, 1, 2], _b[0], _b[-1],
                    colors='#353132', linestyles='dashed')
    _ = axrb.hist(m, bins=_b, weights=sigma_ratio_h, histtype="step", lw=2, color="r")
    axrb.set_ylim(-3, +3)
    axrb.set_ylabel("(exp-kde)/sigma_kde")
    
#     plt.savefig("./data/figs/kde_data_1d_{}.png".format(
#             xlabel[i]), dpi=200)
    plt.show()

# Define the Likelihoods

Here we define our Likelihoods.
We are given a source event occurence (can be GRB, GW, HESE or anything else) at a given position in space and time.
We want to search for a significant contribution of other events, within a predefined region in time and space around the source events.
For this we need to derive the expected signal and background contributions in that frame.

The Likelihood that describes this scenario can be derived from counting statistics.
If we expect $n_S$ signal and $n_B$ background events in the given frame, then the probability of observing $N$ events is given by a poisson pdf:

$$
    P_\text{Poisson}(N\ |\ n_S + n_B) = \mathcal{L}(N | n_S, n_b) = \frac{(n_S + n_B)^{N}}{N!}\cdot \exp{-(n_S + n_B)}
$$

We want to fit for the number of signal events $n_S$ in the frame.
But each event doesn_t have the same probability of contributing to either signal or background, because we don't have that information on a per event basis.
So we include prior information on a per event basis to account for that.

$$
    \mathcal{L}(N | n_S, n_B) = \frac{(n_S + n_B)^{N}}{N!}\cdot \exp{-(n_S + n_B)} \cdot \prod_{i=1}^N P_i
$$

Also the simple poisson pdf above only has one parameter, the total number of events, which can be fit for.
So we need to resolve this degeneracy in $n_S$, $n_B$ by giving additional information.
For that we include a weighted combination of the probability for an event to be signal, denoted by the PDF $S_i$ and for it to background, denoted by $B_i$.
Because the simple counting probabilities are $n_S / (n_S + n_B)$ to count a signal event and likewise $n_B / (n_S + n_B)$ to count a background event we construct the per event prior $P_i$ as:

$$
    P_i = \frac{n_S}{n_S + n_B}\cdot S_i + \frac{n_B}{n_S + n_B}\cdot B_i
        = \frac{n_S \cdot S_i + n_B \cdot B_i}{n_S + n_B}
$$

Note, that for equal probabilities $S_i$ and $B_i$, we simply and up with the normal poisson counting statistic.

Plugging that back into the likelihood we get:

$$
    \mathcal{L}(N | n_S, n_B) = \frac{(n_S + n_B)^{N}}{N!}\cdot \exp{(-(n_S + n_B))} \cdot \prod_{i=1}^N \frac{n_S \cdot S_i + n_B \cdot B_i}{n_S + n_B}
$$

Taking the natrual logarithm to get the log-likelihood we arrive at:

$$
    \ln\mathcal{L}(N | n_S, n_B) = -(n_S + n_B) -\ln(N!) + \sum_{i=1}^N \ln((n_S + n_B) P_i)
$$

If we weight up $n_S$ then every events signal PDF is contributing a bit more than the background pdf.
So the fitter tries to find the combination of $n_S$ and $n_B$ that maximizes the likelihood.

To further simplify, we can use a measured and fixed background expectation rate $\langle n_B\rangle$ and fit only for the number of signal events.
Then we only fit for the number of signal events $n_S$.
Also we are only interested in the number of signal events. 
The fixed background rate can be extracted from data by using the pdf of a larger timescale and average over that (or fit a function) to ensure that local fluctuations don't matter.

Then we end up with our full Likelihood (the denominator in $P_i$ cancels with the term from the poisson PDF):

$$
    \ln\mathcal{L}(N | n_S) = -(n_S + \langle n_B\rangle) -\ln(N!) + \sum_{i=1}^N \ln(n_S S_i + \langle n_B\rangle B_i)
$$

For the test statistic $\Lambda=-2T$ we want to test the hypothesis of having no signal $n_S=0$ vs. the alternative with a free parameter $n_S$:

\begin{align}
    T &= \ln\frac{\mathcal{L}_0}{\mathcal{L}_1}
          = \ln\frac{\mathcal{L}(n_S=0)}{\mathcal{L}{\hat{n}_S}} \\
         &= -(\hat{n}_S + \langle n_B\rangle) -\ln(N!) +
              \sum_{i=1}^N \ln(\hat{n}_S S_i + \langle n_B\rangle B_i) \\
         &\phantom{=} +\langle n_B\rangle +\ln(N!) -
               \sum_{i=1}^N \ln(\langle n_B\rangle B_i) \\
         &= -\hat{n}_S + \sum_{i=1}^N
             \ln\left( \frac{\hat{n}_S S_i}{\langle n_B\rangle B_i} + 1 \right)
\end{align}

The per event PDFs $S_i$ and $B_i$ can depend on arbitrary parameters.
The common choise here is to use a time, energy proxy and spatial proxy depency which has most seperation power:

$$
    S_i(x_i, t_i, E_i) = S_T(t_i) \cdot S_S(x_i) \cdot S_E(E_i) \\ 
    B_i(x_i, t_i, E_i) = B_T(t_i) \cdot B_S(x_i) \cdot B_E(E_i) 
$$

Because the Likelihood only contains ratios of the PDF, we only have to construct functions of the signal to background ratio for each time, spatial and energy distribution.

For the energy PDFs $S_E, B_E$ we use a 2D representation in reconstructed energy and declination because this has the most seperation power (see coenders & skylab models).
The spatial part $S_S, B_S$ is only depending on the distance from source to event, not on the absilute position on the sphere.
The time part $S_T, B_T$ is equivalent to that, only using the distance in time between source event and event.

## Time PDF ratio

Background in uniformly distributed in the time window.
Signal distribtution is falling off gaussian-like at both edges so normalization is different.
So the ratio $S_T / B_T$ is simply the the signal pdf divided by the uniform normalization $1 / (t_1 - t_0)$ in the time frame.

The signal PDFs written out explicitely, where $t_0$ is the source events time and $t$ the events time:

$$
    N \cdot S_T(t, t_0) = \begin{cases}
                     \frac{1}{\sqrt{2\pi}\sigma_T}\exp\left(-\frac{(t-T_0)^2}{2\sigma_T^2}
                     \right)&\quad\mathrm{, if }\ t \in [a, T_0]\\                
                     \frac{1}{\sqrt{2\pi}\sigma_T}&\quad\mathrm{, if }\ t \in [T_0, T_1]\\
                     \frac{1}{\sqrt{2\pi}\sigma_T}\exp\left(-\frac{(t-T_1)^2}
                     {2\sigma_T^2}\right)&\quad\mathrm{, if }\ t \in [T_1, b]\\ 
                    0 &\quad\mathrm{, else}
                  \end{cases}
$$

where $a, b$ are the bounds of the total time window, $T_0, T_1$ are the part, in which the signal is assumed to be uniformly distributed in time and $\sigma_T$ is the width of the gaussian edges.
The gaussian width $\sigma_T$ is as wide as the interval $T_1-T_0$ but constraint to the nearest value in $[2, 30]$ seconds if the frame gets too large or too small.
The total normalization $N$ is given by integrating over $S_T$ in $[a, b]$, resulting in:

$$
    N = \Phi(b) - \Phi(a) + \frac{T_1-T_0}{\sqrt{2\pi}\sigma_T}
$$

where

$$
    \Phi(x) = \int_{-\infty}^{x}\frac{1}{\sqrt{2\pi}\sigma_T}
      \exp\left(-\frac{(t-T_0)^2}{2\sigma_T^2}\right)\mathrm{d}t
$$
the CDF of the gaussian PDF.

The background PDF respectively is simply:

$$
    B_T(t, t_0) = \begin{cases}
                     \frac{1}{b-a}&\quad\mathrm{, if }\ t \in [a, b]\\ 
                    0 &\quad\mathrm{, else}
                  \end{cases}    
$$

To get finite support we truncate the gaussian edges at $n\cdot\sigma_T$.
Though arbitrarliy introduced the concrete cutoff of the doesn't really matter (so say 4, 5, 6 sigma, etc).

This is because in the LLH we get the product of $\langle b_B \rangle B_i$.
A larger cutoff make the normalization of the BG pdf larger, but in the same time makes the number of expected BG event get higher in the same linear fashion.
So as long as we choose a cutoff which ensures that $S \approx 0$ outside, we're good to go.

**Note:** The time PDF doesn't really have seperation power, it more or less acts as a theta function cutting out regions around the source time.
We add a little artificial seperation power for very small time windows by using gaussian edge model.

In [ ]:
def time_soverb(t, t0, dt, nsig):
    """
    Time signal over background PDF.
    
    Signal and background PDFs are each normalized over seconds.
    Signal PDF has gaussian edges to smoothly let it fall of to zero, the
    stddev is dt when dt is in [2, 30]s, otherwise the nearest edge.

    To ensure finite support, the edges are truncated after nsig * dt.

    Parameters
    ----------
    t : array-like
        Times given in MJD for which we want to evaluate the ratio.
    t0 : float
        Time of the source event.
    dt : array-like, shape (2)
        Time window [start, end] in seconds centered at t0 in which the 
        signal pdf is assumed to be uniform.
    nsig : float
        Clip the gaussian edges at nsig * dt
    """
    dt = np.atleast_1d(dt)
    if len(dt) != 2:
        raise ValueError("Timeframe 'dt' must have [start, end] in seconds.")
    if dt[0] >= dt[1]:
        raise ValueError("Interval 'dt' must not be negative or zero.")

    secinday = 24. * 60. * 60.

    # Normalize times from data relative to t0 in seconds
    # Stability: Multiply before subtracting avoids small number rounding?
    _t = t * secinday - t0 * secinday
   
    # Create signal PDF
    # Constrain sig_t to [2, 30]s regardless of uniform time window
    dt_tot = np.diff(dt)
    sig_t = np.clip(dt_tot, 2, 30)
    sig_t_clip = nsig * sig_t
    gaus_norm = (np.sqrt(2 * np.pi) * sig_t)
    
    # Split in def regions gaus rising, uniform, gaus falling
    gr = (_t < dt[0]) & (_t >= dt[0] - sig_t_clip)
    gf = (_t > dt[1]) & (_t <= dt[1] + sig_t_clip)
    uni = (_t >= dt[0]) & (_t <= dt[1])
    
    pdf = np.zeros_like(t, dtype=np.float)
    pdf[gr] = scs.norm.pdf(_t[gr], loc=dt[0], scale=sig_t)
    pdf[gf] = scs.norm.pdf(_t[gf], loc=dt[1], scale=sig_t)
    # Connect smoothly with the gaussians
    pdf[uni] = 1. / gaus_norm
    
    # Normalize signal distribtuion: Prob in gaussians + uniform part
    dcdf = (scs.norm.cdf(dt[1] + sig_t_clip, loc=dt[1], scale=sig_t) -
            scs.norm.cdf(dt[0] - sig_t_clip, loc=dt[0], scale=sig_t))
    norm = dcdf + dt_tot / gaus_norm
    pdf /= norm
    
    # Calculate the ratio
    bg_pdf = 1. / (dt_tot + 2 * sig_t_clip)
    ratio = pdf / bg_pdf
    return ratio

In [ ]:
h = hlp._create_runtime_bins(exp["timeMJD"], goodrun_dict=goodrun_dict,
                             remove_zero_runs=True)

# Make a plot with ratios for different time windows as in the paper
# Arbitrary start date from data
t0 = h["start_mjd"][100]
t0_sec = t0 * secinday

# dt from t0 in seconds, clip at 4 sigma
dts = [[-1, 5], [-5, 50], [-20, 200]]
nsig = 4

# Make t values for plotting in MJD around t0, to fit all in one plot
max_dt, min_dt = np.amax(dts), np.amin(dts)
dt_tot = max_dt - min_dt
clip = np.clip(dt_tot, 2, 30) * nsig
plt_range = np.array([min_dt - clip, max_dt + clip])
t = np.linspace(t0_sec + 1.2 * plt_range[0],
                t0_sec + 1.2 * plt_range[1], 1000) / secinday
_t = t * secinday - t0 * secinday

# Mark event time
plt.axvline(0, 0, 1, c="#353132", ls="--", lw=2)

colors = ["C0", "C3", "C2"]
for i, dt in enumerate(dts):
    # Plot ratio S/B
    SoB = time_soverb(t, t0, dt, nsig)
    plt.plot(_t, SoB, lw=2, c=colors[i],
             label=r"$T_\mathrm{{uni}}$: {:>3d}s, {:>3d}s".format(*dt))
    # Fill uniform part, might look nicely
    # fbtw = (_t > 0) & (_t < dt)
    # plt.fill_between(_t[fbtw], 0, SoB[fbtw], color="C7", alpha=0.1)

# Make it look like the paper plot, but with slightly extended borders, to
# nothing breaks outside the total time frame
plt.xlim(1.2 * plt_range)
plt.ylim(0, 3)
plt.xlabel("t - t0 in sec")
plt.ylabel("S / B")
plt.legend(loc="upper right")
plt.grid(ls="--", lw=1)

# plt.savefig("./data/figs/time_pdf_ratio.png", dpi=200)

plt.show()

## Spatial Pdf

The spatial pdf is holding information on how close the event was to the source position.
Close events are more likely to originate from the source.

To model this behavioure we use a Kent distribution (gaussian correctly normalized on a sphere).

$$
    S_S(x_\mathrm{evt}; x_S, \kappa) = \frac{\kappa}{4\pi \sinh{\kappa}}\cdot \exp(\kappa\cos(\psi))
$$

where $x_\mathrm{evt}$ is the directional vector of the event, $x_S$ is the directional vector of the source an $\kappa$ resembles to the uncertainty in the event reconstruction and is connected with the more familiar $\sigma$ error by.

The connections between $\kappa$ and $\sigma$ is valid up to a $\sigma\approx 40^\circ$ and is given by $\kappa = 1 /\sigma^2$.

Classicaly the background pdf is constructed from data
It is assumed to be uniform in right-ascension and the declination dependence is modeled with a spline fitted to a histogram in sinDec.
Then the PDF is given by:

$$
    B_S(x_\mathrm{evt}) = \frac{1}{2\pi}\cdot p(\sin\delta)
$$

But we already made the work of creating a smooth KDE of our data in logE, declination and sigma.
So we can use that KDE to get the values of our declination distribution.
Because integrating out the KDE is slow, we just use our previous sample from the KDE, bin it finely (quasi continously) and interpolate it with a spline to get also values from in between.
This way we are not dependent on a binning on the data itself, but can use the available validated KDE PDF.

In [ ]:
def spatial_signal(src_ra, src_dec, ev_ra, ev_dec, ev_sig, kent=True):
        """
        Spatial distance PDF between source position(s) and event positions.

        Signal is assumed to cluster around source position(s).
        The PDF is a convolution of a delta function for the localized sources
        and a Kent (gaussian on a sphere) distribution with the events
        positional reconstruction error as width.
        
        Multiplie source positions can be given, to use it in a stacked
        search.
        
        Parameters
        -----------
        src_ra : array-like
            Src positions in equatorial RA in radian: [0, 2pi].
        src_dec : array-like
            Src positions in equatorial DEC in radian: [-pi/2, pi/2].
        ev_ra : array-like
            Event positions in equatorial RA in radian: [0, 2pi].
        ev_dec : array-like
            Event positions in equatorial DEC in radian: [-pi/2, pi/2].
        ev_sig : array-like
            Event positional reconstruction error in radian (eg. Paraboloid).
        
        Returns
        --------
        S : array-like, shape(n_sources, n_events)
            Spatial signal probability for each event and each source.

        """
        # Shape (n_sources, 1), suitable for 1 src or multiple srcs
        src_ra = np.atleast_1d(src_ra)[:, np.newaxis]
        src_dec = np.atleast_1d(src_dec)[:, np.newaxis]

        # Dot product in polar coordinates
        cosDist = (np.cos(src_ra - ev_ra) *
                   np.cos(src_dec) * np.cos(ev_dec) +
                   np.sin(src_dec) * np.sin(ev_dec))
    
        # Handle possible floating precision errors
        cosDist = np.clip(cosDist, -1, 1)
        
        if kent:
            # Stabilized version for possibly large kappas
            kappa = 1. / ev_sig**2
            S = (kappa / (2. * np.pi * (1. - np.exp(-2. * kappa))) *
                 np.exp(kappa * (cosDist - 1. )))
        else:
            # Otherwise use standard symmetric 2D gaussian
            dist = np.arccos(cosDist)
            ev_sig_2 = 2 * ev_sig**2
            S = np.exp(-dist**2 / (ev_sig_2)) / (np.pi * ev_sig_2)
        
        return S
    
def create_spatial_bg_spline(sin_dec, bins=100, range=None, k=3):
    """
    Fit an interpolsating spline to the a histogram of sin(dec).
    
    The spline is fitted to the logarithm of the histogram, to avoid ringing.
    Normalization is done by normalizing the hist.
    
    Parameters
    ----------
    sin_dec : array-like
        Sinus declination coorcinates of each event, [-1, 1].
    bins : int or array-like
        Binning passed to `np.histogram`. (default: 100)
    range : array-like
        Lower and upper boundary for the histogram. (default: None)
    k : int
        Order of the spline. (default: 3)
        
    Returns
    -------
    spl : scipy.interpolate.InterpolatingSpline
        Spline object interpolating the histogram. Must be evaluated with
        sin(dec) and exponentiated to give the correct values.
        Spline is interpolating outside it's definition range.
    """
    hist, bins = np.histogram(sin_dec, bins=bins, 
                              range=range, density=True)
    
    if np.any(hist <= 0.):
        estr = ("Declination hist bins empty, this must not happen. Empty " +
                "bins: {0}".format(np.arange(len(bins) - 1)[hist <= 0.]))
        raise ValueError(estr)
    elif np.any((sin_dec < bins[0]) | (sin_dec > bins[-1])):
        raise ValueError("Data outside of declination bins!")

    mids = 0.5 * (bins[:-1] + bins[1:])
    return sci.InterpolatedUnivariateSpline(mids, np.log(hist), k=k, ext=0)

def spatial_background(ev_sin_dec, sindec_log_bg_spline):
    """
    Calculate the value of the backgournd PDF for each event from a previously
    created spline, interpolating the declination distribution of the data.
    
    Parameters
    ----------
    ev_sin_dec : array-like
        Sinus Declination coordinates of each event, [-1, 1].
    sindec_log_bg_spline : scipy.interpolate.InterpolatingSpline
        Spline returning the logarithm of the bg PDF at given sin_dec values.
    
    Returns
    -------
    B : array-like
        The value of the background PDF for each event.
    """
    return 1. / 2. / np.pi * np.exp(sindec_log_bg_spline(ev_sin_dec))


def spatial_SoB(src_ra, src_dec, ev_ra, ev_dec, ev_sig,
                sindec_log_bg_spline, kent=True):
    S = spatial_signal(src_ra, src_dec, ev_ra, ev_dec, ev_sig, kent)
    B = spatial_background(ev_sin_dec, sindec_log_bg_spline)
    
    SoB = np.zeros_like(S)
    B = np.repeat(B[np.newaxis, :], repeats=S.shape[0], axis=0)
    m = B > 0
    SoB[m] = S[m] / B[m]

    return SoB

### Signal PDF

In [ ]:
def plot_dec_vs_signal(S, ev_dec, src_ra, src_dec, weights, ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    # Plot signal per source for each event
    for i, (sra, sdec) in enumerate(zip(src_ra, src_dec)):
        ax.plot(np.rad2deg(ev_dec), S[i], ls="-")
        ax.plot(np.rad2deg(sdec), -10, "k|")

    # Simulate a simple stacking, one weight per source
    ax.plot(np.rad2deg(ev_dec), np.sum(weights * S, axis=0) / np.sum(weights),
             ls="--", c=dg, label="stacked")

    ax.set_xlim([-1 + smin, smax + 1])
    ax.set_xlabel("DEC in °")
    ax.set_ylabel("Signal pdf")
    ax.legend(loc="upper right")
    return ax

# Simulate a simple case: 5 src and the events are in the same range, but with
# tighter spacing
smax = 5
smin = -5
step = 2

src_ra = np.deg2rad(np.arange(smin, smax + step, step))
src_dec = np.deg2rad(np.arange(smin, smax + step, step))

ev_ra = np.deg2rad(np.linspace(smin, smax, 1000))
ev_dec = np.deg2rad(np.linspace(smin, smax, 1000))
ev_sig = np.deg2rad(np.ones_like(ev_ra))

S = spatial_signal(src_ra, src_dec, ev_ra, ev_dec, ev_sig, kent=True)  

weights = np.arange(1, len(src_dec) + 1)[:, np.newaxis]
_ = plot_dec_vs_signal(S, ev_dec, src_ra, src_dec, weights)
plt.show()

# Now with the real data. Sort first in dec to show with nice lines
idx = np.argsort(exp["dec"])
ev_ra = exp["ra"][idx]
ev_dec = exp["dec"][idx]
# ev_sig = np.deg2rad(np.ones_like(ev_ra))
ev_sig = exp["sigma"][idx]

S = spatial_signal(src_ra, src_dec, ev_ra, ev_dec, ev_sig, kent=True)

weights = np.ones_like(weights)
ax = plot_dec_vs_signal(S, ev_dec, src_ra, src_dec, weights)
ax.set_yscale("log")
ax.set_ylim(1, 1e4)
plt.show()

### Background PDF

In [ ]:
# Assign model from CV, which has already evaluated adaptive kernels
kde_inj = json2kde("data/awKDE_CV/CV10_glob_bw_alpha_EXP_IC86I_" +
                   "CUT_sig.ll.20_PARS_diag_True_pass2.json")

# Generate some BG samples to compare to the original data hist
nsamples_kde = int(1e7)
bg_samples = kde_inj.sample(nsamples_kde)

# Plot in degrees and in sinDec
_sam_kde = np.vstack((bg_samples[:, 0],
                      np.sin(bg_samples[:, 1]),
                      np.rad2deg(bg_samples[:, 2]))).T

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 4))

# First finely binned KDE. Show the data in the same binning to see the diff
bins = np.linspace(-1, 1, 100)
h, b, _ = axl.hist(_sam_kde[:, 1], bins=bins, normed=True, alpha=0.5)
h, b = np.histogram(_sam_kde[:, 1], bins=bins, density=True)
kde_spl = create_spatial_bg_spline(_sam_kde[:, 1], bins=bins)

_sin_dec = np.linspace(-1, 1, 1000)
pdf = np.exp(kde_spl(_sin_dec))
axl.plot(_sin_dec, pdf, lw=2)
axl.set_title("BG PDF from KDE: PDF constructed with fine bins")

# Now classic with coarse binned data
bins = np.linspace(-1, 1, 20 + 1)
sin_dec = np.sin(exp["dec"])
h, b, _ = axr.hist(sin_dec, bins=bins, normed=True, alpha=0.5)
spl = create_spatial_bg_spline(sin_dec, bins=bins)

pdf = np.exp(spl(_sin_dec))
axr.plot(_sin_dec, pdf)
axr.set_title("BG PDF from data")

# Quickly integrate BG pdf to check norm is OK (increased subdvivision lim)
I = scint.quad(spatial_background, -1, 1, args=(kde_spl), limit=100)[0]
print("Area under all sky BG PDF is : ", 2. * np.pi * I)

### Signal over Background

In [ ]:
# Make srcs across the dec range. SoB should follow the sinDec BG
# distribtuion. With a single source we couldn't see that, because it drops
# to zero far from the src position
bg_sin_dec = _sam_kde[:, 1]
smin, smax, step = -90, +90, 10

src_ra = np.deg2rad(np.arange(smin, smax + step, step))
src_dec = np.deg2rad(np.arange(smin, smax + step, step))

ev_ra = np.deg2rad(np.linspace(smin, smax, 1000))
ev_dec = np.deg2rad(np.linspace(smin, smax, 1000))
ev_sin_dec = np.sin(ev_dec)
ev_sig = np.deg2rad(np.ones_like(ev_ra))

weights = np.arange(1, len(src_dec) + 1)[:, np.newaxis]

fig, ((axtl, axtr), (axbl, axbr)) = plt.subplots(2, 2, figsize=(12, 10))

# Signal only
S = spatial_signal(src_ra, src_dec, ev_ra, ev_dec, ev_sig, kent=True)  
_ = plot_dec_vs_signal(S, ev_dec, src_ra, src_dec, weights, ax=axtl)
axtl.set_xlim(-90, 90)

# Background only
bins = 100
h, b, _ = axl.hist(bg_sin_dec, bins=bins, normed=True, alpha=0.5)
h, b = np.histogram(bg_sin_dec, bins=bins, density=True)
_sin_dec = np.linspace(-1, 1, 1000)
pdf = np.exp(kde_spl(_sin_dec))
axbl.plot(np.rad2deg(np.arcsin(_sin_dec)), pdf, lw=2, label="pdf")
axbl.set_ylim(0, 1)
# 1 / BG PDF on second axis
axbl2 = axbl.twinx()
axbl2.plot(np.rad2deg(np.arcsin(_sin_dec)), 1. / pdf, c="C1",
           lw=2, label="1/pdf")
axbl2.set_ylim(0, 6)
axbl.set_xlabel("DEC in °")
axbl.set_xlim(-90, 90)
axbl.legend(loc="upper left")
axbl2.legend(loc="upper center")

# SoB on example + BG PDF
SoB = spatial_SoB(src_ra, src_dec, ev_ra, ev_dec, ev_sig, kde_spl, kent=True)  
weights = np.arange(1, len(src_dec) + 1)[:, np.newaxis]
_ = plot_dec_vs_signal(SoB, ev_dec, src_ra, src_dec, weights, ax=axtr)
axtr.plot(np.rad2deg(np.arcsin(_sin_dec)), pdf, lw=3, label="BG pdf", c=dg)
axtr.set_xlim(-90, 90)
axtr.set_yscale("log")
axtr.set_ylim(0.1, 1e5)
axtr.legend(loc="upper left")

# Now with the real data. Sort first in dec to show with nice lines + BG PDF
idx = np.argsort(exp["dec"])
ev_ra = exp["ra"][idx]
ev_dec = exp["dec"][idx]
ev_sin_dec = np.sin(ev_dec)
ev_sig = exp["sigma"][idx]
# ev_sig = np.deg2rad(np.ones_like(ev_ra))  # To match the simple example

SoB = spatial_SoB(src_ra, src_dec, ev_ra, ev_dec, ev_sig, kde_spl, kent=True)

_ = plot_dec_vs_signal(SoB, ev_dec, src_ra, src_dec, weights, ax=axbr)
axbr.plot(np.rad2deg(np.arcsin(_sin_dec)), pdf, lw=3, label="BG pdf", c="C0")
axbr.set_yscale("log")
axbr.set_ylim(0.1, 1e5)
axbr.legend(loc="upper left")

plt.show()

## Energy-Space Pdf

This will be the same in skylab and the first time we need a MC set.
Make equally binned 2D histograms in logE and sinDec, then take the ratio.
Because of the equal binning, the normalization is automatically correct.
Then fit a 2D spline to it which gives the signal to background ratio directly.

Here we use again a KDE fitted both to data.
This way we can sample more events in the sparsely populated areas and obtain a broader ratio distribution.
Because we can't use the sklearn KDE for weighted samples we use a normal histogram for the MC, which has more event anyway so the problem is not so urgent.

Where data is missing either use background MC or conservatively use the highest ratio where data is available also at positions, where no data is present.
This is only relevant for signal injection, because on data we have the ratio defined everywhere, where data is by definition.

In [ ]:
def get_bg_sample_from_kde(nsamples_kde=int(1e7)):
    # Assign model from CV, which has already evaluated adaptive kernels
    kde_inj = json2kde("data/awKDE_CV/CV10_glob_bw_alpha_EXP_IC86I_" +
                       "CUT_sig.ll.20_PARS_diag_True_pass2.json")

    bg_samples = kde_inj.sample(nsamples_kde)

    # Plot in degrees and in sinDec
    _sam_kde = np.vstack((kde_sam[:, 0],
                          np.sin(kde_sam[:, 1]),
                          np.rad2deg(kde_sam[:, 2]))).T

    # Return sinDec and logE
    return _sam_kde[:, 1], _sam_kde[:, 0]

In [ ]:
def get_bg_sample_from_kde(nsamples_kde = int(1e7)):
    print("Sampling from BG KDE")
    # KDE CV is running on cluster and pickles the GridSearchCV
    fname = "./data/kde_cv/KDE_model_selector_20_exp_IC86_I_followup_2nd_pass.pickle"
    with open(fname, "rb") as f:
        model_selector = pickle.load(f)

    kde = model_selector.best_estimator_
    bw = model_selector.best_params_["bandwidth"]
    print("Best bandwidth : {:.3f}".format(bw))

    # We maybe just want to stick with the slightly overfitting kernel to
    # be as close as possible to data
    OVERFIT = True
    if OVERFIT:
        bw = 0.075
        kde = skn.KernelDensity(bandwidth=bw, kernel="gaussian", rtol=1e-8)
    print("Used bandwidth : {:.3f}".format(bw))

    # KDE sample must be cut in sigma before fitting, similar to range in hist
    _exp = exp[exp["sigma"] <= np.deg2rad(5)]

    fac_logE = 1.5
    fac_dec = 2.5
    fac_sigma = 2.

    _logE = fac_logE * _exp["logE"]
    _sigma = fac_sigma * np.rad2deg(_exp["sigma"])
    _dec = fac_dec * _exp["dec"]

    # Fit KDE best model to background sample
    kde_sample = np.vstack((_logE, _dec, _sigma)).T
    kde.fit(kde_sample)

    # Generate some BG samples to compare to the original data hist.
    # Use more statistics, histograms get normalized and we want the best estimate
    # for the pdf
    bg_samples = kde.sample(n_samples=nsamples_kde)

    # Restore the orignal scaling and cut away spillovers from the finite width
    bg_logE = bg_samples[:, 0] / fac_logE
    bg_dec = bg_samples[:, 1] / fac_dec
    bg_sigma = bg_samples[:, 2] / fac_sigma

    m = (bg_dec > -np.pi / 2.) & (bg_dec < np.pi / 2.)
    m = m & (bg_sigma > 0 )

    bg_logE = bg_logE[m]
    bg_dec = bg_dec[m]
    bg_sindec = np.sin(bg_dec)
    bg_sigma = np.deg2rad(bg_sigma[m])
    
    return bg_sindec, bg_logE

In [ ]:
# Prepare the MC data, signal weighted to astro unbroken power law
gamma = 2.
# No flux norm, because we normalize anyway
mc_w = mc["ow"] * mc["trueE"]**(-gamma)

# Make 2D hist from data KDE and from MC, use the MC binning
mc_sindec = np.sin(mc["dec"])
mc_logE = mc["logE"]
bins = [50, 40]
rnge = [[-1, 1], [1, 10]]
mc_h, bx, by = np.histogram2d(mc_sindec, mc_logE, bins=bins, range=rnge,
                              weights=mc_w, normed=True)

b = [bx, by]

MODE = "DATA"  # "DATA", "KDE_SAM", "KDE_INT"
if MODE == "DATA":
    bg_logE = exp["logE"]
    bg_sindec = np.sin(exp["dec"])
    bg_h, _, _ = np.histogram2d(bg_sindec, bg_logE, bins=b,
                                range=rnge, normed=True)
elif MODE == "KDE_SAM":
    bg_sindec, bg_logE = get_bg_sample_from_kde(int(2e7))
    bg_h, _, _ = np.histogram2d(bg_sindec, bg_logE, bins=b,
                                range=rnge, normed=True)
elif MODE == "KDE_INT":
    _bins = np.load("data/1d_integrate_kde/logE_sinDec_bins_50x50.npy")
    vals = np.load("data/1d_integrate_kde/logE_sinDec_int_50x50.npy")
    mids = get_binmids(_bins)
    xx, yy  = map(np.ravel, np.meshgrid(mids[0], mids[1]))
    bg_h, _, _ = np.histogram2d(xx, yy, bins=_bins, weights=vals,
                                normed=True, range=rnge)
    # Turn around to have sinDec vs logE like in the other examples
    bg_h = bg_h.T
    # KDE_INT is not so good, because it falls too quickly. Need to clip it
    bg_h = np.clip(bg_h, 1e-10, 1)
    
# 3 cases:
#   - Data & MC: Calculate the ratio
#   - No data or no MC: Assign nearest value in energy bin
#   - No data and no MC: Assign any value (eg 1), these are never accessed
# Get logE value per bin in entrie histogram
m = get_binmids(b)

# Fill value: 1) min/max for low/hig edge or 2) nearest in column
FILLVAL = "MINMAX"  # "COL" | "MINMAX"

# This assumes at least one valid point in one sinDec slice
m1 = (bg_h > 0) & (mc_h > 0)
SoB = np.ones_like(bg_h) * -1  # Init with unphysical value
SoB[m1] = mc_h[m1] / bg_h[m1]
SOBmin, SoBmax = np.amin(SoB[m1]), np.amax(SoB[m1])

# In each energy bin assign nearest value to bins with no data or no MC
for i in np.arange(bins[0]):
    bghi = bg_h[i]  # Get sinDec slice
    mchi = mc_h[i]
    _m = (bghi <= 0) | (mchi <= 0)  # All invalid points
    # Only fill missing logE border values and then proceed to interpolation

    # First lower edge (argmax stops at first True, argmin at first False)
    low_first_invalid_id = np.argmax(_m)
    if low_first_invalid_id == 0:
        # Set lower edge with first valid point from bottom
        low_first_valid_id = np.argmin(_m)
        if FILLVAL == "COL":
            SoB[i, 0] = SoB[i, low_first_valid_id]
        elif FILLVAL == "MINMAX":
            SoB[i, 0] = np.amin(SoB[m1])

    # Repeat with turned around array for upper edge
    hig_first_invalid_id = np.argmax(_m[::-1])
    if hig_first_invalid_id == 0:
        # Set lower edge with first valid point from bottom
        hig_first_valid_id = len(_m) - 1 - np.argmin(_m[::-1])
        if FILLVAL == "COL":
            SoB[i, -1] = SoB[i, hig_first_valid_id]
        elif FILLVAL == "MINMAX":
            SoB[i, -1] = np.amax(SoB[m1])
        
    # Interpolate in each slice over missing entries
    _m = SoB[i] > 0
    x = m[1][_m]
    y = SoB[i, _m]
    fi = sci.interp1d(x, y, kind="linear")
    SoB[i] = fi(m[1])

# These do never occur, so set them to 1to be identified quickly in the plot
m4 = (bg_h <= 0) & (mc_h <= 0)
SoB[m4] = 1.

# Now fit a spline to the ratio
SoB_spl = sci.RegularGridInterpolator(m, np.log(SoB), method="linear",
                                      bounds_error=False, fill_value=0.)

In [ ]:
# Coenders style sindec vs logE
m = get_binmids(b)
xx, yy = map(np.ravel, np.meshgrid(*m))

fig, ax = plt.subplots(2, 2, figsize=(12, 10))

(axtl, axtr), (axbl, axbr) = ax

# Data
_, _, _, img = axtl.hist2d(xx, yy, bins=b, weights=bg_h.T.flatten(),
                         norm=LogNorm())
axtl.set_title("Exp events : {}".format(len(exp)))
caxtl = split_axis(axtl, cbar=True)
plt.colorbar(cax=caxtl, mappable=img)

# MC
_, _, _, img = axtr.hist2d(xx, yy, bins=b, weights=mc_h.T.flatten(),
                         norm=LogNorm())
axtr.set_title("Signal. gamma = {:.1f}".format(gamma))
caxtr = split_axis(axtr, cbar=True)
plt.colorbar(cax=caxtr, mappable=img)

# Ratio hist
cnorm = max(np.amin(SoB), np.amax(SoB))  # coenders: 1e-3, 1e3
_, _, _, img = axbl.hist2d(xx, yy, bins=b, weights=SoB.T.flatten(),
                         norm=LogNorm(), cmap="coolwarm",
                         vmin=1. / cnorm, vmax=cnorm)
axbl.set_title("Signal over background".format(gamma))
caxbl = split_axis(axbl, cbar=True)
plt.colorbar(cax=caxbl, mappable=img)

# Ratio spline
x = np.linspace(*rnge[0], num=500 + 1)
y = np.linspace(*rnge[1], num=500 + 1)
XX, YY = np.meshgrid(x, y)
xx, yy = map(np.ravel, [XX, YY])
gpts = np.vstack((xx, yy)).T
zz = np.exp(SoB_spl(gpts))
ZZ = zz.reshape(XX.shape)
# Plotting with hist creates strange effects... Use pcolormesh instead
img = axbr.pcolormesh(XX, YY, ZZ, norm=LogNorm(), cmap="coolwarm",
                    vmin=1. / cnorm, vmax=cnorm)
axbr.set_title("Spline interpolation".format(gamma))
caxbr = split_axis(axbr, cbar=True)
plt.colorbar(cax=caxbr, mappable=img)

# plt.savefig("./data/figs/energy_ratio_spline_minmaxfill.png", dpi=200)
plt.show()

# Sensitivity

Sensitivity calculation example using a simple rayleigh distribution (positive values, single shape parameter).

In [ ]:
# Set background TS value against which the alternative is compared. This is
# connected to an alpha value = percentage of pdf lying right of H0 TS_val
TS_val = 2

# beta value = percentage of pdf lying right of TS_val
beta = 0.9

def rayleigh_quantiles(TS_val, p):
    """
    Get the scale factor for a given probability and TS value.
    https://www.npmjs.com/package/distributions-rayleigh-quantile
    """
    return TS_val / np.sqrt(-np.log((1. - p)**2))

## What we want to have

We have a TS value that we define our threshold for (is equivalent to setting an alpha for the type I error).
We need to find how many signal like events we need to inject to get the transformed TS with a fraction beta above the predefined alpha value (or TS value) of the null hypothesis.
Here we do that by adapting the scale parameter of the rayleigh distribution which is the same but easier to follow.

In [ ]:
# For a showcase let's make some distrubtions of the alternative hypothesis
scales = np.array([3., 4., 5., 6.])
nsamples = 10000
H1 = [np.random.rayleigh(scale=s, size=nsamples) for s in scales]

# Their 1-beta fraction lies further to the right with incresing scale
i = 0
xmax = 10
x = np.linspace(0, xmax, 200)
bins = np.linspace(0, xmax, 50)
for H1i in H1:
    plt.hist(H1i, bins=bins, alpha=0.3, color="C{}".format(i))
    plt.hist(H1i, bins=bins, color="C{}".format(i), histtype="step", lw=2)
    plt.axvline(np.percentile(H1i, (1 - beta) * 100), 0, 1, lw=3,
                color="C{}".format(i),
                label="Scale: {:.1f}".format(scales[i]))
    plt.plot(x, nsamples * np.diff(bins)[0] *
             scs.rayleigh.pdf(x, loc=0, scale=scales[i]),
             color="C{}".format(i), lw=2, ls="--")
    i += 1

plt.axvline(TS_val, 0, 1, color="k", ls="--", lw=2)
plt.xlim(0, xmax)
plt.xlabel("TS value")
plt.ylabel("Counts")
plt.legend(loc="upper right")
plt.show()

For the rayleigh distribution we even can get the analytic solution to the problem:

In [ ]:
true_scale = rayleigh_quantiles(TS_val, p=1. - beta)
print("Analytic scale factor for TS value {:.1f} is: {:.3f}".format(
      TS_val, true_scale))

# For a showcase let's make some distrubtions of the alternative hypothesis
nsamples = 10000
H1 = np.random.rayleigh(scale=true_scale, size=nsamples)

# Their 1-beta fraction lies further to the right with incresing scale
plt.hist(H1, bins=bins, alpha=0.3, color="C7")
plt.hist(H1, bins=bins, color="C7", histtype="step", lw=2)
plt.axvline(np.percentile(H1, (1. - beta) * 100), 0, 1, lw=3, color="C7")
plt.plot(x, nsamples * np.diff(bins)[0] *
         scs.rayleigh.pdf(x, loc=0, scale=true_scale),
         color="C7", lw=2, ls="--")

plt.axvline(TS_val, 0, 1, color="k", ls="--", lw=2)
plt.xlim(0, xmax)
plt.xlabel("TS value")
plt.ylabel("Counts")
plt.show()

## Fitting it

Now instead of guessing where the right scale is, we try to fit it.
For this we choose a scale, create a sample and check how close the desired 1-beta percentile is to the TS value we defined.
From the plot above, we already see, that the sought after scale should be between 4 and 5.

The more we sample the more accurate the result is.
Here we know the distribution and could get exact results, but in reality we often have to sample to get the distribution.

### Root Finder

**This can be implemented easily as an automated method, as seen below**

Here the fit is a root search.
The target function is defined as the difference between the targeted TS values and the percentile of the current sample.

In [ ]:
def do_trials(scale, nsamples=10000):
    return np.random.rayleigh(scale=scale, size=nsamples)

def fit_root(scale, nsamples):
    sam = do_trials(scale, nsamples)
    return TS_val - np.percentile(sam, 100 * (1. - beta))

In [ ]:
res = sco.brentq(fit_root, 0., 10., args=(10000))
print("TS value: {:.3f}".format(res))

See how much we spread around the true value with that method.

We see:

1. The spread gets smaller with more trials per sample, as expected
2. The estimator is pretty unbiased

Note: Setting xtol and rtol has not a real influence on the spread.
It still get really close to the break condition by chance.

In [ ]:
res = [sco.brentq(fit_root, 0., 10., args=(1000), xtol=1e-10, rtol=1e-10)
       for i in range(1000)]
plt.hist(res, bins=30)
plt.axvline(rayleigh_quantiles(TS_val, p=0.1), 0, 1,
            ls="--", color="C1", lw=3)

### Least Squares Scan with Parabola Fit

**This is more or less a manual method, where we select the support point by hands**

Alternatively we can define a least squares fit function with the squared distance and do a minimization.
But the simple minimization doesn't work because the samples are different even for very close scale parameters due to the finite sample size.
So instead we scant he loss function and fit a parabole (exact shape) to the scan points.
The minimum of the parabola is then the sought after scale parameter.
Also an error estimate can be given using the covariance of the fit parameter.

Note: The parabola might only be exact in a small range around the minimum, because the percentile might be a very nonlinear function in the scale parameter.
Nevertheless the approcimation should be good close around the minimum.

In [ ]:
def do_trials(scale, nsamples=10000):
    return np.random.rayleigh(scale=scale, size=nsamples)

def fit_ls(scale, nsamples):
    sam = do_trials(scale, nsamples)
    return (TS_val - np.percentile(sam, 100 * (1. - beta)))**2

def parabola(x, a, b):
    return a * (x - b)**2

In [ ]:
# This sticks to the seed, because the function always changes due to the
# sampling in each step
for seed in [1., 2., 3., 4.]:
    res = sco.minimize(fit_ls, seed, bounds=[[0., 10.]], args=(10000))
    print("Seed is {:.1f} -> TS value: {:.3f}".format(seed, res.x[0]))

So we do a scan and fit a parabola:

In [ ]:
# Fit a parabola to the sample points to make the estimate more robust
# scale = np.linspace(0.1, 4., 5)
scale = np.arange(10)
y = np.array([fit_ls(s, nsamples=1000) for s in scale])
bf, cov = sco.curve_fit(parabola, scale, y, p0=[1., 1.],
                      bounds=[[0., 0.], [np.inf, np.inf]])

x = np.linspace(0., 10., 100)
y_fit = parabola(x, *bf)

plt.plot(scale, y)
plt.plot(x, y_fit, ls="--")
pb_min = bf[1]
plt.axvline(pb_min, 0, 1, ls="--", color="C1")

stddev =  np.sqrt(cov[1, 1])
print("TS value: {:.3f} ± {:.3f} ({:.1f}%)".format(
    pb_min, stddev, stddev / pb_min * 100))

plt.xlabel("TS value")
plt.ylabel("Least Squares Value")
plt.show()

See how much we spread around the true value with that method.

We see:

1. More samples, less spreads, as before
2. Seems to be unbiased
3. Spread seems as big as in root finder method, see below for comparison

In [ ]:
res = []
scale = np.linspace(0.1, 10., 10)
ntrials = 1000
nsampels = 1000

for i in range(ntrials):
    y = np.array([fit_ls(s, nsamples=nsamples) for s in scale])
    bf, _ = sco.curve_fit(parabola, scale, y, p0=[1., 1.],
                          bounds=[[0., 0], [np.inf, np.inf]])
    pb_min = bf[1]
    res.append(pb_min)
    
plt.hist(res, bins=15)
plt.axvline(rayleigh_quantiles(TS_val, p=0.1), 0, 1,
            ls="--", color="C1", lw=3)

Compare to root finder trials:

In [ ]:
res_bq = [sco.brentq(fit_root, 0., 10., args=(nsamples),
                     xtol=1e-10, rtol=1e-10) for i in range(ntrials)]

In [ ]:
h_pb, b, _ = plt.hist(res, bins=30, histtype="step", label="Parabola", lw=3)
h_bq, _, _ = plt.hist(res_bq, bins=b, histtype="step", label="Roots BQ", lw=3)

plt.errorbar(get_binmids(b), h_pb, yerr=np.sqrt(h_pb), c="C0", fmt="none")
plt.errorbar(get_binmids(b), h_bq, yerr=np.sqrt(h_bq), c="C1", fmt="none")

plt.axvline(rayleigh_quantiles(TS_val, p=0.1), 0, 1,
            ls="--", color="C7", lw=3, label="True")

plt.axvline(np.mean(res), 0, 1, color="C0", ls="--", label="Mean PB")
plt.axvline(np.mean(res_bq), 0, 1, color="C1", ls="--", label="Mean BQ")

plt.legend(loc="upper right")
plt.tight_layout()
plt.show()

### Fitting the CDF

**This would also be a manual method. We choose some injection values and do trials for each to get supprt point for the CDF fit. Problem here is, what is the exact shape of the CDF?**

Instead of fitting the loss function asnd looking for its minimum, we could also note the percentile above a certain threshold for each set of trials with a different expectation.
Then we can fit a CDF directly to the percentile vs mu plot.
Also we can reuse a set of trials for every answer we want to have, sensitivitys, disc. potential etc. by altering the percentile definiton afterwards.

In [ ]:
def do_trials(scale, nsamples=10000):
    return np.random.rayleigh(scale=scale, size=nsamples)

def percentile(sample, thresh):
    """
    Returns the percentage of sample points above a threshold.
    
    Parameters
    ----------
    sample : array-like
        Data values on which the percentile is calculated.
    thresh : float
        Threshold in x-space to calculate the percentile against.
    
    Returns
    -------
    perc : float
        Percentile in [0, 1], fraction of data point > val.
    err : float
        Estimated relative error on the percentile.
    """
    sample = np.atleast_1d(sample)
    n = len(sample)
    perc = np.sum(sample > thresh) / n
    z = 1.  # 1 sigma error (gaussian approximation)
    err = z * np.sqrt(perc * (1. - perc) / n)
    return perc, err

In [ ]:
ntrials = 10000
scale_scan = np.arange(0.5, 7, 0.5)

percs = []
errors = []
for si in scale_scan:
    perc, err = percentile(do_trials(si, nsamples=ntrials), TS_val)
    percs.append(perc)
    errors.append(err)

def chi2_cdf_fit(x, df, loc, scale):
    return scs.chi2.cdf(x, df, loc, scale)

def exp_fit(x, a, b, c):
    return 1. - a * np.exp(-(x / b)**c)


chi2_res, chi2_err = sco.curve_fit(f=chi2_cdf_fit, xdata=scale_scan,
                                   ydata=percs, sigma=errors)
exp_res, exp_err = sco.curve_fit(f=exp_fit, xdata=scale_scan,
                                 ydata=percs, sigma=errors)

In [ ]:
x = np.linspace(0, 7, 200)
y_chi2 = chi2_cdf_fit(x, *chi2_res)
y_exp = exp_fit(x, *exp_res)

plt.errorbar(scale_scan, percs, yerr=errors, fmt="o")
plt.plot(x, y_chi2, label="chi2({:.2f}, {:.2f}, {:.2f})".format(*chi2_res))
plt.plot(x, y_exp, label="1-{:.2f}*exp(-(x/{:.2f})^{:.2f})".format(*exp_res))

plt.xlabel("scale")
plt.ylabel("CDF")
plt.title("Rayleigh. Percentile above TS={:.1f} for {} trials each".format(
    TS_val, ntrials))

# Also plot the true percentile vs scale function
true_p = 1. - scs.rayleigh.cdf(x=TS_val, scale=x)
plt.plot(x, true_p, ls="--", color="k", label="True CDF(scale)")

# At which scale is chi2 with beta above TS_val?
est_scale = scs.chi2.ppf(beta, *chi2_res)
print("Estimated true scale: {:.2f}".format(est_scale))
print("True scale: {:.2f}".format(rayleigh_quantiles(p=1.-beta,
                                                     TS_val=TS_val)))
plt.axhline(beta, 0, 1, ls="--", color="k")
plt.axvline(est_scale, 0, 1, ls="--", color="k")

plt.axhline(1, 0, 1, ls="-", color="C7")
plt.xlim(0, None)
plt.ylim(0, 1.1)

plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

Also test how strong the spread is for repeated experiments.

We see:

1. For too few trials, the fit often does not converge
2. It is overestimating the true scale, because the fit is not good for small scales. Problem here is, in general we don't know the true CDF as in the parabola case, where we had the quadratic loss function (here we can simply plot the truth because we know the Rayleigh CDF)
3. Spread seems to be as big as in the previous cases

The good thing is, that we always can check, if the fit is good and choose support points where we need them to be.
Also we should consider not using the fit, if the estimated parameter falls in a region where the fit is describing the data badly.
So we should guess the location of the correct value with a scane, sample densly around it and try to find a CDF that is fitting good in this region, just to interpolate.
Could use a spline too.

In [ ]:
nscans = 1000
ntrials = 10000
scale_scan = np.arange(0.5, 7, 0.5)
est_scales = []

for i in range(nscans):
    percs = []
    errors = []
    for si in scale_scan:
        perc, err = percentile(do_trials(si, nsamples=ntrials), TS_val)
        percs.append(perc)
        errors.append(err)

    chi2_res, chi2_err = sco.curve_fit(f=chi2_cdf_fit, xdata=scale_scan,
                                       ydata=percs, sigma=errors)
    est_scales.append(scs.chi2.ppf(beta, *chi2_res))
    
# We see some failing fits for too few trials. Otherwise the spread is
# similar to other methods
_ = plt.hist(est_scales, bins=50)
plt.axvline(rayleigh_quantiles(p=1-beta, TS_val=TS_val), 0, 1, color="C1")

### Iterative Poisson Reweighting

**This Method is used in skylab and csky.**

The idea is to do a coarse scan in the scale parameter first to get an idea where the minimum is.
From there on we iteratively generate trials around the current best fitting poisson mean mu to enhance the statistics, so we can reuse all trials made so far.

So we do:

1. Make a coarse grid scan until the 1-beta quantile is close enough to the desired TS value to get a good starting seed for the fit..
2. For the corrsponding number of events to inject we start the iterative minimization.
   We iterate until the error on the quantile is lower than a given tolerance.
3. We get the error from the Wald (gaussian) approximation to the binomial error interval on the estimator $\hat{p}$, because we have a fixed number of trials and estimate the number of trials falling in one of two regions TS > TS_val.
   The loss function is a least squares loss with the distance from the current quantile TS value to the desired TS value.
3. With the current best fit poisson mean number of injected events we do more batches of trials by drawing the actual number of events to inject from a poisson distribution with the best fit mean until our precision is good enough.

**Read here for weighted Wald Binomial error:**
1. https://stats.stackexchange.com/questions/159204/how-to-calculate-the-standard-error-of-a-proportion-using-weighted-data
2. https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval#Normal_approximation_interval

**And to not forget the reference, weighted Poisson errors:** http://www.hep.uiuc.edu/e687/memos/weight_err.PS

In [ ]:
def make_ns_poisson_weights(mu, ns):
    """
    The values of ns are drawn from poisson PDF with a specific mu.
    To reuse trials generated from a different mu, we can reweight the ns
    values to match the current mu.
    
    Parameters
    ----------
    mu : float
        Possion expectation value, >= 0.
    ns : array-like
        ns values from various trials.
        
    Returns
    -------
    weights : array-like
        Weights to reweights ns values to the poissnian with expectaion mu.
    hist : array-like
        Number of events falling in each ns bin.
    """
    if mu < 0.:
        raise ValueError("Poisson expectation 'mu' must be >= 0.")
    # Bin ns values to assign proper weight
    hist = np.bincount(ns)
    nbins = len(hist)
    bins = np.arange(nbins)
    # Get probability per bin for current mu
    pmf = scs.poisson.pmf(bins, mu)
    # Get weights, split equally per bin
    weights = np.zeros(nbins)
    mask = hist > 0.
    weights[mask] = pmf[mask] / hist[mask]
    # Reassign weight to per trial ns
    idx = np.digitize(ns, bins, right=True)
    return weights[idx], hist

def get_weighted_percentile(x, val, w=None):
    """
    Calculate the weighted percentile of data `x` with weight `w`.    

    This calculates the amount of data in `x` lying under the threshold
    `val` (analogue to np.percentile).
    The relativ error is estimated from weighted counting statistics:
    
        sum(w[x <= val]) / sum(w)
    
    Parameters
    ----------
    x : array-like
        Data values on which the percentile is calculated.
    val : float
        Threshold in x-space to calculate the percentile against.
    w : array-like
        Weight for each data point. If None, all weights are assumed to be 1.
        (default: None)
    
    Returns
    -------
    perc : float
        Percentile in [0, 1], fraction of data point > val.
    err : float
        Estimated relative error on the percentile.
    """
    x = np.asarray(x, dtype=float)
    
    if w is None:
        w = np.zeros_like(x) + 1. / len(x)
    elif np.sum(w**2) == 0:
        raise ValueError("Squared weight sum is zero, so weights are zero.")
        
    # Get weighted percentile
    mask = x > val
    perc = np.sum(w[mask]) / np.sum(w)
    # Binomial error on weighted percentile in Wald approximation
    err = np.sqrt(perc * (1. - perc) * np.sum(w**2))
    
    return perc, err

def performance(ts_val, beta, trial_func, mu0=None, ntrials=100,
                tol_perc_err=5e-3, tol_mu_rel=1e-3, maxloops=100,
                verb=False):
    """
    Iteratively search for the best fit `mu`, so that a fraction `beta` of
    the scaled PDF lies above the background test statistic value `ts_val`.

    Performance search on a PDF parameter `mu` which is the expectation value
    for a poisson PDF via a second variable defining a test statistic which
    is directly influenced by the choice of `mu`.
        
    Parameters
    ----------
    ts_val : float
        Test statistic value of the BG distribution, which is connected to
        the alpha value (Type I error).
    beta : float
        Fraction of alternative hypothesis PDF that should lie right of the
        `ts_val`.
    trial_func : callable
        Function returning a single trial depending on the value of the
        current parameter `mu`.
    mu0 : float, optional
        Seed value to begin the minimization at. If None a region close to
        the minimum is searched for automatically. If explicitely given, it
        must be >= 0. (default: None)
    ntrials : int, optional
        How many new trials to make per new iteration. (default: 100)
    tol_perc_err, tol_mu_rel : float, optional
        The iteration stops when BOTH of the following conditons are met:
        
        - The error on the estimated percentile for the current best fit
          ``mu`` is ``errors[-1] <= tol_perc_err`` AND
        - The relative difference in the best fit ``mus`` is
          ``abs(mus[-1]-mus[-2])/mus[-1]<= tol_mu_rel``.

        Furthermore the conditions must be met in BOTH the last AND second to
        last trial loops to avoid a break on accidental fluctuations.
        (default: tol_perc_err: 5e-3, tol_mu_rel: 1e-3)
    maxloops : int, optional
        Break the minimization process after this many loops with ntrials
        trials each. (default: 100)
    verb : bool, optional
        If ``True``print convergence message during fit. (default: False)
        
    Returns
    -------
    res : dict
        Result dictionary with keys:
        
        - "mu_bf": Best fit mu, equal to mu[-1].
        - "mu": List of visited mu values during minimization.
        - "ts": List of generated TS values during minimization.
        - "ns": List of generared ns values during minimization.
        - "err": List of errors on the weighted TS percentile per iteration.
        - "perc": List of estimated TS percentiles per iteration.
        - "nloops": Number of iterations needed to converge.
        - "ninitloops": Number of initial scan iterations done.
        - "lastfitres": scipy.optimize.OptimizeResult of the last fit.
        - "converged": Boolean, if ``True`` fit converged within maxloops.
    """
    def loss(mu, ns, ts):
        """
        Logged least squares loss for percentile distance to beta. No
        gradient is returned, because is has a pole at the minimum and the
        minimizer doesn't like that.
        
        Parameters
        ----------
        mu : float
            Current expectation value for poisson PDF.
        ns : array-like
            ns values from all trials done so far.
        ts : array-like
            Test statistic values from all trials done so far.
        
        Returns
        -------
        loss : float and array-like
            Value of the loss function at the current mu.
        """
        # Reweight TS trials using the poisson statistics of ns
        w, _ = make_ns_poisson_weights(mu=mu, ns=ns)
        perc, _ = get_weighted_percentile(x=ts, val=ts_val, w=w)
        return np.log10((perc - beta)**2)
        
    def append_batch_of_trials(n, mu, ns, ts):
        """Do n trials and append result to ns and ts arrays"""
        for ni in np.random.poisson(mu, size=n):
            ns = np.append(ns, ni)
            ts = np.append(ts, trial_func(ni))
        return ns, ts
    
    def get_perc_and_err(mu, ns, ts):
        """
        Get the percentile and its relative error for all trials under
        the current best fit mu.
        """
        w, _ = make_ns_poisson_weights(mu=mu, ns=ns)
        perc, err = get_weighted_percentile(x=ts, val=ts_val, w=w)
        return perc, err
    
    # Keep track of progress
    mus = np.array([], dtype=np.float)
    ts = np.array([], dtype=np.float)
    ns = np.array([], dtype=np.int)
    errors = np.array([], dtype=np.float)
    percs = np.array([], dtype=np.float)
    n_init_loops = 0
    n_loops = 0
    converged = False
    
    # If no seed given, start initial scan to get close to the minimum
    dmu = 5.  # Must be handtuned to the problem...
    if mu0 is None:
        n_init_trials = 10  # Not too few but also not too many for 1st scan
        def frac_over_tsval(ts):
            """Fraction of 'n_init_trials' last trials above 'ts_val'"""
            if len(ts) < n_init_trials:
                return 0.
            return (np.sum(ts[-n_init_trials:] > ts_val) /
                    n_init_trials)
        
        mu = 1.        
        stop = False
        while not stop:
            ns, ts = append_batch_of_trials(n_init_trials, mu, ns, ts)
            
            # Save progress
            mus = np.append(mus, mu)
            # Err est. is not reliable here, too few trials, so use worst err
            perc, _ = get_perc_and_err(mu, ns, ts)
            errors = np.append(errors, 1.)
            percs = np.append(percs, perc)       
            
            # Overshoot in the last two batches to have trials above best fit
            if n_init_loops > 2:
                stop = ((frac_over_tsval(ts) > beta) and
                        (frac_over_tsval(ts[:-n_init_trials]) > beta))
                
            mu += dmu
            n_init_loops += 1
           
        if verb:
            print("Made {} intitial scan loops with {} trials total".format(
                n_init_loops, len(ts)))

    elif mu0 < 0.:
        raise ValueError("Seed `mu0` must be >= 0.")
    else:
        # Init and do first batch of trials
        mu = mu0
        mus = np.append(mus, mu)
        errors = np.append(errors, 1.)
        percs = np.append(percs, -1.)
        ns, ts = append_batch_of_trials(ntrials, mu, ns, ts)

    # Process minimizer loop until last two rel. error are below tolerance
    stop = False
    while n_loops <= 2 or not stop:
        # Make new batch of trials
        ns, ts = append_batch_of_trials(ntrials, mu, ns, ts)
           
        # Now fit the poisson expectation by reusing all (reweighted) trials
        # Bounds: 90% CL central interval around best mu
        bl, bu = scs.poisson.interval(0.90, mu)
        bounds = [bl, max(bu, 1.)]  # Avoid [0, 0] for small mu
        # Do a seed scan prior to fitting to avoid local minima
        seeds = np.arange(*bounds)
        seed = seeds[np.argmin([loss(mui, ns, ts) for mui in seeds])]
               
        res = sco.minimize(loss, [seed], bounds=[bounds], args=(ns, ts),
                           jac=False, method="L-BFGS-B",
                           options={"ftol": 100, "gtol": 1e-8})
        mu = res.x
        perc, err = get_perc_and_err(mu, ns, ts)
              
        # Make some manual tweaks to help the minimizer: (credit: mrichman)
        oldmu = mus[-1]
        # New fit is suddenly more than 50% above old fit: Truncate change
        if np.abs(mu - oldmu) / oldmu > 0.5:
            if mu > oldmu:
                mu = 1.5 * oldmu
            else:
                mu = 0.5 * oldmu
            err = errors[-1]  # Make sure we definitely do another trial
        # Fit is identically to previous fit
        if mu == oldmu:
            mu = 1.1 * oldmu  # Choose larger mu: conservative -> more flux
            err = errors[-1]

        # Save the progress
        mus = np.append(mus, mu)
        errors = np.append(errors, err)
        percs = np.append(percs, perc)
        n_loops += 1
               
        # Error conditions must match in the last and last to last trials
        if n_loops > 2:
            mu_rel_err1 = np.abs(mus[-1] - mus[-2]) / mus[-1]
            mu_rel_err2 = np.abs(mus[-2] - mus[-3]) / mus[-2]
            if ((mu_rel_err1 <= tol_mu_rel) and
                    (mu_rel_err2 <= tol_mu_rel) and
                    (errors[-1] < tol_perc_err) and
                    (errors[-2] < tol_perc_err)):
                if verb:
                    print("Break: below tol_mu_rel and tol_perc_err.")
                converged = True
                stop = True

        if n_loops == maxloops:
            if verb:
                print("Manual break after {} loops with {} main ".format(
                      n_loops, n_loops * ntrials) + "trials: Reached " +
                      "`maxloops` loops.")
            break
        
    return {"mu_bf": mus[-1], "mus": mus, "ts": ts, "ns": ns, "err": errors,
            "perc": percs, "nloops": n_loops, "ninitloops": n_init_loops,
            "lastfitres": res, "converged": converged}

#### Test reweighting

Show that the reweighting works.

In [ ]:
mu = 3.9
# mu = 0.  # Uncomment to verify mu = 0 works too
xmax = 2 * np.ceil((mu + 1))
mids = np.arange(0.5, xmax, 1.)
ns = np.random.randint(0, xmax, 10)
w, nevts = make_ns_poisson_weights(mu, ns)

x = np.arange(0, xmax + 1)
y = scs.poisson.pmf(x, mu)

fig, ax = plt.subplots(1, 1)
ax.hist(ns, bins=np.arange(xmax + 1), weights=w)
ax.plot(np.r_[x[0], x], np.r_[0, y], drawstyle="steps-post",
        label="$\mu={:.1f}$".format(mu))

# Show bin content on 2nd axis
ax2 = ax.twiny()
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(mids)
ax2.set_xticklabels(["{:d}".format(ni) for ni in nevts])

# Lines to bins helping to track the 2nd axis down
for m in mids:
    ax.axvline(m, 0, 1, ls="--", color="C7", zorder=-1, alpha=0.5)
ax.fill_between(np.r_[x[0], x], 0, np.r_[0, y], color="w", step="post")

ax.set_xlabel("k")
ax2.set_xlabel("Events per bin")
ax.set_ylabel("Probability")

ax.legend(loc="best")

plt.tight_layout()

Get weighted percentiles and compare unweighted case to numpy percentile function.

In [ ]:
mu = 5
xmax = 2 * np.ceil((mu + 1))
ns = np.arange(3 * xmax, dtype=int)
w, nevts = make_ns_poisson_weights(mu, ns)

perc_val = 2

print("Sorted ns values:")
print(np.sort(ns))

print("Requested quantile at ns={}".format(perc_val))

print("\nWeighted percentile: {:.3f}\nRel. Error: {:.3f}".format(
    *get_weighted_percentile(ns, val=perc_val, w=w)))

print("\n\nNow with weights all ones (Should be close to numpy):\n")
alpha = np.linspace(0, 100, 11)
for a in alpha:
    np_val = np.percentile(ns, a, interpolation="lower")
    print(" Get a numpy.percentile at alpha = {:.1f}: {:.2f}".format(
        a / 100., np_val))
    print(" Numpy value put back in the function: {:.2f}".format(
        1 - get_weighted_percentile(ns, val=np_val, w=None)[0]))
    print("")


#### Test error estimate

Use the known Rayleigh distribution for that, so we can use the exact position of the percentile.
Generate trials and construct the percentile and error intervall.
Then count how many times the intervalls hit the correct percentile.
They should do so in the correct fraction alpha as they were constructed with.

Note:

The Wald approximation only works, if the estimated percentile is not close to 0 or 1.
If we have to few events to test, we are more likely to end up with the stimated p close to 0 or 1 even if we test for a beta fartger away of 0 or 1.

In [ ]:
niter = 10000
ntrials = 100  # See how it breaks down with ntrials < ~30
true_scale = rayleigh_quantiles(TS_val, 1. - beta)

print("Doing {:d} times {:d} trials with ".format(niter, ntrials) +
      "true scale = {:.2f} ({:.1f}% over TS val of {:.1f})".format(
          true_scale, beta * 100, TS_val))

perc, err = [], []
contains_true = []
for i in range(niter):
    sample = np.random.rayleigh(scale=true_scale, size=ntrials)
    pi, ei = get_weighted_percentile(sample, val=TS_val, w=None)
    perc.append(pi)
    err.append(ei)
    if (beta <= pi + ei) & (beta >= pi - ei):
        contains_true.append(True)
    else:
        contains_true.append(False)
        
plt.errorbar(np.arange(niter), perc, yerr=err, fmt="none")
l = "Coverage = {:.2f}%".format(100. * np.sum(contains_true) / niter)
plt.axhline(beta, 0, 1, ls="--", color="C1", label=l)
plt.title(r"{:d} times {:d} trials. {} ($1\sigma$)".format(
    niter, ntrials, "Poisson Error" if poisson_error else "Binomial Error"))
plt.legend()

# plt.savefig("data/figs/sensitivity_error_coverage_{}.png".format(
#     "poisson" if poisson_error else "binomial"), dpi=200)

print("Coverage = {:.2f}%".format(100. * np.sum(contains_true) / niter))

Here we test the standard poisson interval on the estimate of the expectation mu for completeness.

The case of a weighted histogram translate as follows:
We have a single weighted measurement, so the estimated expectaion is sum of weights.
Using the formula for variance we get the error on that by using sum of squared weights.

For multiple trials we divide by the number of trials as usual and the error decreases with sqrt(N).
Similar to the binomial case the error interval is only reliable if the expectation value is high.

In [ ]:
niter = 1000
# If the true mu is high the number of trials don't matter. The estimate on
# mu gets better and the error decreases, but the coverage is always correct
ntrials = 1
true_mu = rayleigh_quantiles(TS_val, 1. - beta) * 100
poisson_error = True
alpha = 1. - 0.68  # 1 sigma

print("Doing {:d} times {:d} trials with ".format(niter, ntrials) +
      "true mu = {:.2f} ({:.1f}% over TS val of {:.1f})".format(
          true_mu, beta * 100, TS_val))

mus, err = [], []
contains_true = []
for i in range(niter):
    sample = np.random.poisson(true_mu, size=ntrials)
    mu_est = np.mean(sample)
    ei = np.sqrt(mu_est / ntrials)
    err.append(ei)
    mus.append(mu_est)
    if (true_mu <= mu_est + ei) & (true_mu >= mu_est - ei):
        contains_true.append(True)
    else:
        contains_true.append(False)
        
plt.errorbar(np.arange(niter), mus, yerr=err, fmt=".",
             markersize=1)
l = "Coverage = {:.2f}%".format(100. * np.sum(contains_true) / niter)
plt.axhline(true_mu, 0, 1, ls="--", color="C1", label=l)
plt.title(r"{:d} times {:d} trials. {} ($1\sigma$)".format(
    niter, ntrials, "Poisson Error" if poisson_error else "Binomial Error"))
plt.legend()

# plt.savefig("data/figs/sensitivity_error_coverage_{}.png".format(
#     "poisson" if poisson_error else "binomial"), dpi=200)

print("Coverage = {:.2f}%".format(100. * np.sum(contains_true) / niter))

#### Test iterative fit

Fit a single example and plot some performance curves of the fit.

In [ ]:
%%time
def get_one_trial(n):
    """Wrapper to convert ns to scale parameter"""
    scale = n / 100.
    return np.random.rayleigh(scale=scale, size=1)

ntrials = 100
mu0 = None  # Performs better with intial scan because we have more stats
tol_perc_err = 5e-3
tol_mu_rel = 1e-3
maxloops = 100

res = performance(ts_val=TS_val, beta=beta, trial_func=get_one_trial,
                  ntrials=ntrials, mu0=mu0, tol_perc_err=tol_perc_err,
                  tol_mu_rel=tol_mu_rel, maxloops=maxloops)

mu = res["mu_bf"]
mus = res["mus"]
ns = res["ns"]
ts = res["ts"]
percs = res["perc"]
nloops = res["nloops"]

true_scale = rayleigh_quantiles(TS_val, p=1. - beta)
print("Analytic scale factor for TS value " +
      "{:.1f} and beta {:.2f} is: {:.3f}".format(TS_val, beta, true_scale))
print("Best scale: {:.3f}".format(mu / 100.))
print("Last fit res:", res["lastfitres"])
print("Total trials: {:d}".format(len(ns)))

Plot fit summary over the iterations

In [ ]:
mu_bf = res["mu_bf"]
mus = res["mus"]
ns = res["ns"]
ts = res["ts"]
true_scale = rayleigh_quantiles(TS_val, p=1. - beta)

fig, [[axtl, axtr], [axbl, axbr]] = plt.subplots(2, 2, figsize=(12, 8))

# Plot the scale evolution
axtl.plot(res["mus"] / 100.)
axtl.axhline(true_scale, 0, 1, color="C1", ls="--", label="True scale")
axtl.axvline(res["ninitloops"] - 1, 0, 1, color="C7", ls="--",
             label="Init loops")
axtl.set_title("scale after iteration")
axtl.set_xlabel("Iteration i")
axtl.set_ylabel("Scale")
axtl.legend(loc="best")

# Plot the rel/abs error evolution
axtr.plot(res["err"], color="C0", label="perc err")
axtr.plot(np.abs(np.diff(mus)) / mus[1:], color="C1", label="mu diff abs")
axtr.axhline(tol_perc_err, 0, 1, color="C0", ls="--", label="tol perc")
axtr.axhline(tol_mu_rel, 0, 1, color="C1", ls="--", label="mu diff tol")
axtr.axvline(res["ninitloops"] - 1, 0, 1, color="C7", ls="--",
             label="Init loops")
axtr.set_title("Performance after ith Iteration")
axtr.set_xlabel("Iteration i")
axtr.set_yscale("log", nonposy="clip")
axtr.legend(loc="best")

# Plot all drawn, unweighted ns values
_ = axbl.hist(ns, bins=200)
axbl.set_title("all ns unweighted")
axbl.axvline(100 * true_scale, 0, 1, color=dg, ls="--",
             label="100 * (true scale)")
axbl.set_xlabel("ns")
axbl.set_ylabel("Count")
axbl.legend(loc="best")

# Plot all drawn, weighted test statistic values
bins = np.arange(0., 3 * np.ceil(mu_bf) / 100., 0.25)
w, _ = make_ns_poisson_weights(mu_bf, ns)
perc, _ = get_weighted_percentile(ts, TS_val, w)
axbr.set_title("all ts weighted, perc = {:.6f}".format(perc))
_ = axbr.hist(ts, bins=bins, weights=w)
axbr.axvline(TS_val, 0, 1, color="C1", ls="--", label="TS val")
axbr.axvline(true_scale, 0, 1, color=dg, ls="--", label="True scale")
axbr.set_xlabel("TS")
axbr.set_ylabel("PDF")
axbr.legend(loc="best")

plt.tight_layout()
# plt.savefig("data/figs/performance_poisson_reweight.png", dpi=150)
plt.show()

Plot loss function for final iteration and best fit location

In [ ]:
mu_bf = res["mu_bf"]
ns = res["ns"]
ts = res["ts"]
true_scale = rayleigh_quantiles(TS_val, p=1. - beta)

# Repeat function inside `performance` to see the steps
def loss(mu, ns, ts):
    w, _ = make_ns_poisson_weights(mu=mu, ns=ns)
    perc, _ = get_weighted_percentile(x=ts, val=TS_val, w=w)
    return np.log10((perc - beta)**2)

bounds = scs.poisson.interval(0.90, mu_bf)
seeds = np.arange(*bounds)
seed = seeds[np.argmin([loss(mui, ns, ts) for mui in seeds])]

# Scan the loss function
mu_low = 1.
mu_hig = 2. * mu_bf
mu_rng = np.arange(mu_low, mu_hig).astype(np.float)
f = []
for mui in mu_rng:
    f.append(loss(mui, ns, ts))

fig, ax = plt.subplots(1, 1)
ax.plot(mu_rng, f, color="C0", label="loss")
ax.axvline(mu_bf, 0, 1, ls="-", color="C1", label="best fit")
ax.axvline(bounds[0], 0, 1, ls=":", color=dg, label="fit bounds")
ax.axvline(bounds[1], 0, 1, ls=":", color=dg)
ax.axvline(true_scale * 100, 0, 1, ls="--", color=dg, label="truth")
plt.xlabel("100 * scale")
plt.ylabel("Loss function")
plt.legend(loc="lower left")
plt.tight_layout()
plt.show()

#### Test many trials and see best fit distribution

In [ ]:
%%time
def get_one_trial(n):
    """Wrapper to convert ns to scale parameter"""
    scale = n / 100.
    return np.random.rayleigh(scale=scale, size=1)

ntrials = 100
tol_perc_err = 1e-3
tol_mu_rel = 1e-3
maxloops = 100

nexps = 100
res = []
for i in tqdm(range(nexps)):
    resi = performance(ts_val=TS_val, beta=beta, trial_func=get_one_trial,
                       ntrials=ntrials, mu0=None, tol_perc_err=tol_perc_err,
                       tol_mu_rel=tol_mu_rel, maxloops=maxloops, verb=False)
    res.append(resi)
    
mu_bfs = np.array([resi["mu_bf"] for resi in res])
errs = np.array([resi["err"][-1] for resi in res])
percs = np.array([resi["perc"][-1] for resi in res])

In [ ]:
true_scale = rayleigh_quantiles(TS_val, p=1. - beta)
fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))
iteration = np.arange(nexps)
mu_errs = np.sqrt(mu_bfs)
true_mu = 100 * true_scale

# Coverages: They may not make much sense, because the parameters are not
# estimated from a random process but are fitted parameters
# If the fit is good, more and more intervals hit the true value, but not
# as expected by real coverage.
perc_cov = (beta <= percs + errs) & (beta >= percs - errs)
perc_cov = np.sum(perc_cov) / nexps
mu_cov = (true_mu <= mu_bfs + mu_errs) & (true_mu >= mu_bfs - mu_errs)
mu_cov = np.sum(mu_cov) / nexps

axl.errorbar(iteration, percs, yerr=errs, fmt=".")
axl.axhline(beta, 0, 1, color="C1", ls="--")
axl.set_title("Coverage: {:.2%}".format(perc_cov))

axr.errorbar(iteration, mu_bfs, yerr=mu_errs, fmt=".")
axr.axhline(true_scale * 100, 0, 1, ls="--", color="C1")
axr.set_title("Coverage: {:.2%}".format(mu_cov))

plt.tight_layout()
plt.show()

#### Animate it!

Same code as above but modified so it animates all the single trials and fits in between.

In [ ]:
def get_one_trial(n):
    """Wrapper to convert ns to scale parameter"""
    scale = n / 100.
    return np.random.rayleigh(scale=scale, size=1)


def performance(ts_val, beta, trial_func,
                mu0=None, ntrials=100, eps_rel=1e-3):
    def loss(mu, ns, ts):
        w, _ = make_ns_poisson_weights(mu=mu, ns=ns)
        perc, _ = get_weighted_percentile(x=ts, val=ts_val, w=w)
        return np.log10((perc - beta)**2)
    
    mus = np.array([], dtype=np.float)
    ts = np.array([], dtype=np.float)
    ns = np.array([], dtype=np.int)
    errors = np.array([], dtype=np.float)
    n_loops = 0
    
    if mu0 is None:
        mu = -0.9            
        n_init_trials = 100  
        def frac_over_tsval(ts):
            """Fraction of 'n_init_trials' last trials above 'ts_val'"""
            if len(ts) < n_init_trials:
                return 0.
            return (np.sum(ts[-n_init_trials:] > ts_val) /
                    n_init_trials)
        
        n_init_loops = 1
        while frac_over_tsval(ts) < beta:
            mu += 2.
            for n in np.random.poisson(mu, size=n_init_trials):
                ns = np.append(ns, n)
                ts = np.append(ts, trial_func(n))
            n_init_loops += 1
            
        print("Made {} intitial scan loops with {} trials total".format(
            n_init_loops, len(ts)))
    elif mu0 < 0.:
        raise ValueError("Seed `mu0` must be >= 0.")
    else:
        mu = mu0
        
    mus = np.append(mus, mu)
    errors = np.append(errors, 1)
    for n in np.random.poisson(mu, size=ntrials):
            ns = np.append(ns, n)
            ts = np.append(ts, trial_func(n))
        
    while True:
        for n in np.random.poisson(mu, size=ntrials):
            ns = np.append(ns, n)
            ts = np.append(ts, trial_func(n))
            
        bounds = np.percentile(ns[-ntrials:], [5, 95])
        seeds = np.arange(bounds)
        seed = seeds[np.argmin([loss(mui, ns, ts) for mui in seeds])]

        res = sco.minimize(loss, [seed], bounds=[bounds], args=(ns, ts),
                           jac=False, method="L-BFGS-B")       
        
        mu = res.x
        w, _ = make_ns_poisson_weights(mu=mu, ns=ns)
        perc, err = get_weighted_percentile(x=ts, val=ts_val, w=w)
              
        if mu == mus[-1]:
            mu = 0.9 * mus[-1]      

        mus = np.append(mus, mu)
        errors = np.append(errors, err)
        n_loops += 1
        
        yield {"mu_bf": mus[-1], "mus": mus, "ts": ts, "ns": ns,
                "err": errors, "nloops": n_loops}


def animate(j):
    def loss(mu, ns, ts):
        w, _ = make_ns_poisson_weights(mu=mu, ns=ns)
        perc, _ = get_weighted_percentile(x=ts, val=TS_val, w=w)
        return np.log10((perc - beta)**2)

    plt.clf()
    print("Anim {}".format(j))
    
    # Add a new trial
    res = next(perf_gen)
    mu = res["mu_bf"]
    ns = res["ns"]
    ts = res["ts"]
    true_scale = rayleigh_quantiles(TS_val, p=1. - beta)
    
    rnge = [0, 800]
    f = np.zeros(np.diff(rnge), dtype=float)
    mu_rng = np.arange(*rnge)

    for i, mui in enumerate(mu_rng.astype(float)):
        fi = loss(mui, ns, ts)
        f[i] = fi
    
    # Reproduce next step before running new trial to visulaize
    bounds = np.percentile(ns[-1000:], [5, 95])
    seed = np.argmin([loss(mui, ns, ts) for mui in np.arange(*bounds)])
    min_res = sco.minimize(loss, [seed], bounds=[bounds], args=(ns, ts),
                           jac=False, method="L-BFGS-B")

    plt.plot(mu_rng, f)
    plt.axhline(0, 0, 1, ls="--", color="k")
    plt.axvline(true_scale * 100, 0, 1, ls="--", color="k")
    plt.axvline(mu, 0, 1, ls="-", color="k")
    plt.axvline(seed, 0, 1, ls="--", color="C1")
    plt.axvline(min_res.x, 0, 1, ls="-", color="C1")
    plt.axvline(bounds[0], 0, 1, ls=":", color="k")
    plt.axvline(bounds[1], 0, 1, ls=":", color="k")
    return

ntrials = 1000
perf_gen = performance(ts_val=TS_val, beta=beta, trial_func=get_one_trial,
                       ntrials=ntrials, mu0=None)

nanim = 30
fig, ax = plt.subplots(1, 1)
ani = animation.FuncAnimation(fig, animate, range(nanim))

Writer = animation.writers["ffmpeg"]
writer = Writer(fps=2, bitrate=2000)
ani.save(filename="vid.mp4", writer=writer)

print("Done!")

## csky comparison

In [ ]:
def get_poisson_weights (mu, ns):
    """
    Weight trials to Poisson distribution.
    """
    bin_counts = np.bincount (ns)
    bin_ns = np.arange (bin_counts.size)
    if mu > 0:
        bin_ps = scs.poisson (mu).pmf (bin_ns)
    else:
        bin_ps = np.r_[1., np.zeros (bin_counts.size - 1)]
    bin_ws = np.zeros (bin_counts.size)
    mask = bin_counts > 0
    bin_ws[mask] = bin_ps[mask] / bin_counts[mask]
    ws = bin_ws[np.searchsorted (bin_ns, ns)]
    return ws

def get_poisson_mu (ns, tss, thresh, beta, mu_guess):
    """
    Get best guess of mu for ts > thresh at rate ``beta`` with error estimate.
    """
    mask = np.array (tss) > thresh
    def log10miss2 (mu):
        ws = get_poisson_weights (mu, ns)
        f = np.sum (ws[mask]) / np.sum (ws)
        return np.log10 ((f - beta)**2)

    #bounds = np.percentile (ns[ns > 0], [10, 80])
    if mu_guess < 0:
        mu_guess = np.median (ns)
    bounds = [np.sqrt (mu_guess), mu_guess**2]
    mu, f, info = sco.fmin_l_bfgs_b (
        log10miss2, [mu_guess],
        bounds=[bounds],
        approx_grad=True
    )
    warnflag = info['warnflag']
    if False and warnflag:
        note = ' ({})'.format (info['task']) if warnflag == 2 else ''
        print (22 * ' ' + '- note: warnflag = {}{}'.format (warnflag, note))
    mu = mu[0]
    ws = get_poisson_weights (mu, ns)
    err = np.sqrt (np.sum (ws[mask]**2)) / np.sum (ws)
    return mu, err, info

def get_n_sig_generic (ts, beta, get_one_trial,
                       n_sig_start=-1,
                       n_batch=500, tol=4e-3,
                       log=True, full_output=False):
    """
    Obtain an n_sig reaching the ``ts`` threshold ``beta`` fraction of trials.
    """
    n_batch = int (n_batch)
    assert 0 < tol < 1

    # bookkeeping setup
    n_batchs = [0]
    errs = [1]
    ns, tss = np.array ([], dtype=int), np.array ([])

    # make a first guess
    first_tss = np.array ([])
    n_sig = 0 if n_sig_start <= 0 else max (5, n_sig_start - 5)
    err =errs[-1]
    i = 0
    while 1. * np.sum (first_tss > ts) / 10 < beta:
        n_sig += 5
        i += 10
        first_tss = np.array ([])
        for i_trial in range (10):
            n = np.random.poisson (n_sig)
            first_tss = np.r_[first_tss, get_one_trial (n)]
        f = 1. * np.sum (np.array (first_tss) > ts) / len (first_tss)

    n_sigs = [n_sig]

    # loop until stable convergence seems like a realistic possibility
    while len (n_batchs) <= 2 or (errs[-1] > tol or errs[-2] / errs[-1] > 10):
        new_ns, new_tss = np.array ([], dtype=int), np.array ([])

        # try n_sig distributed about best fit so far
        for n in np.random.poisson (n_sig, n_batch):
            new_ns = np.r_[new_ns, n]
            new_tss = np.r_[new_tss, get_one_trial (n)]
        # note this batch of trials
        ns = np.r_[ns, new_ns]
        tss = np.r_[tss, new_tss]
        n_sig_last = n_sig
        err_last = err
        # fit poisson distribution
        n_sig, err, info = get_poisson_mu (ns, tss, ts, beta, n_sig)
        # slow your roll if the change is > 50%
        if np.abs (n_sig_last - n_sig) / n_sig_last > .5:
            n_sig = n_sig_last * (1.5 if n_sig > n_sig_last else .5)
            err= err_last
        # more bookkeeping
        n_sigs.append (n_sig)
        errs.append (err)
        n_batchs.append (n_batchs[-1] + n_batch)
        # more fine adjustments
        if n_sigs[-2] == n_sigs[-1]:
            n_sig *= .9
        if (errs[-1] < tol and len (n_batchs) <= 2):
            n_sig *= 1.05
            errs[-1] = tol + np.finfo (float).eps
        if not errs[-1]:
            n_sig *= 2

    if full_output:
        return dict (n_trial=np.array (n_batchs),
                     n_sig=np.array (n_sigs),
                     err=np.array (errs),
                     ns=np.array (ns),
                     tss=np.array (tss),
                     ws=get_poisson_weights (n_sig, ns),
                     n_sig_best=n_sigs[-1])
    else:
        return n_sig
    
def get_one_trial(n):
    scale = n / 100.
    return do_trials(scale=scale, nsamples=1)

In [ ]:
%%time
res_csky = get_n_sig_generic(TS_val, beta, get_one_trial, n_sig_start=-1,
                             n_batch=500, tol=4e-3, full_output=True)

true_scale = rayleigh_quantiles(TS_val, p=1-beta)
print("Analytic scale factor for TS value {:.1f} is: {:.3f}".format(
      TS_val, true_scale))
print("Best scale: {:.3f}".format(res_csky["n_sig_best"] / 100.))
print("Used {} trials in total.".format(res_csky["n_trial"][-1]))

In [ ]:
mu_bf = res_csky["n_sig_best"]
ns = res_csky["n_sig"]
ts = res_csky["tss"]
true_scale = rayleigh_quantiles(TS_val, p=1. - beta)

fig, [[axtl, axtr], [axbl, axbr]] = plt.subplots(2, 2, figsize=(12, 8))

# Plot the scale evolution
axtl.plot(ns / 100.)
axtl.axhline(true_scale, 0, 1, color="C1")
axtl.set_title("scale after iteration")
axtl.set_xlabel("iteration i")

# Plot the rel/abs error evolution
axtr.plot(np.log10(res_csky["err"]), color="C0", label="rel")
axtr.axhline(np.log10(eps_rel), 0, 1, color="C0", ls="--")
axtr.set_title("log10(uncertainty) after iteration")
axtr.set_xlabel("iteration i")
axtr.legend(loc="best")

# Plot all drawn, unweighted ns values
# Here not so good, because the single ns values are not saved
fig.delaxes(axbl)
# _ = axbl.hist(ns, bins=50)
# axbl.set_title("all ns unweighted")
# axbl.set_xlabel("ns")

# Plot all drawn, weighted test statistic values
bins = np.arange(0., 3 * np.ceil(mu_bf) / 100., 0.25)
w = res_csky["ws"]
perc, _ = get_weighted_percentile(ts, TS_val, w)
axbr.set_title("all ts weighted, perc = {:.2f}".format(perc))
_ = axbr.hist(ts, bins=bins, weights=w)
axbr.axvline(TS_val, 0, 1, color="C1", ls="-")
axbr.axvline(true_scale, 0, 1, color="k", ls="--")
axbr.set_xlabel("TS")

plt.tight_layout()
# plt.savefig("data/figs/performance_poisson_reweight_csky.png", dpi=150)
plt.show()